In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import LabelEncoder
from itertools import product

# 데이터 불러오기
file_path = r"C:\Users\m\Desktop\머신러닝 사용 데이터\피처엔지니어링한통합데이터\피처엔지니어링일단다한통합데이터.csv"
data = pd.read_csv(file_path)

# ------------------------
# 🏢 사용자 입력 받는 함수
# ------------------------
def get_user_input():
    상권_코드_명 = input("상권 코드를 입력하세요: ")
    서비스_업종_코드_명 = input("업종명을 입력하세요: ")
    return 상권_코드_명, 서비스_업종_코드_명

# 사용자 입력 받기
상권_코드_명, 서비스_업종_코드_명 = get_user_input()

# ------------------------
# 📊 데이터 전처리
# ------------------------

# 1. 데이터 필터링 (사용자 입력한 상권 & 업종만 선택)
data = data[(data["상권_코드_명"] == 상권_코드_명) & (data["서비스_업종_코드_명"] == 서비스_업종_코드_명)]

# 2. 기준_년분기_코드 기준 정렬 (시간순 정렬)
data = data.sort_values(by="기준_년분기_코드")

# 3. 2023년 4분기까지 학습, 2024년 데이터는 검증용
train_data = data[data["기준_년분기_코드"] <= 20234]
test_data = data[data["기준_년분기_코드"] > 20234]

# 4. 요일 & 시간대 개별 피처로 학습을 위한 데이터 변환
요일_columns = ["월요일", "화요일", "수요일", "목요일", "금요일", "토요일", "일요일"]
시간대_columns = ["00~06", "06~11", "11~14", "14~17", "17~21", "21~24"]

# 요일별 매출 건수 / 매출 금액 데이터 변환
요일_매출건수 = {요일: f"{요일}_매출_건수" for 요일 in 요일_columns}
요일_매출금액 = {요일: f"{요일}_매출_금액" for 요일 in 요일_columns}

# 시간대별 매출 건수 / 매출 금액 데이터 변환
시간대_매출건수 = {시간대: f"시간대_{시간대}_매출_건수" for 시간대 in 시간대_columns}
시간대_매출금액 = {시간대: f"시간대_{시간대}_매출_금액" for 시간대 in 시간대_columns}

# ------------------------
# 🏆 머신러닝 모델 학습
# ------------------------

# 요일 학습을 위한 데이터셋 구성
X_train_요일 = train_data[list(요일_매출건수.values())]
y_train_요일 = train_data[list(요일_매출금액.values())].mean(axis=1)

# 시간대 학습을 위한 데이터셋 구성
X_train_시간대 = train_data[list(시간대_매출건수.values())]
y_train_시간대 = train_data[list(시간대_매출금액.values())].mean(axis=1)

# 모델 학습 (랜덤 포레스트 사용)
model_요일 = RandomForestRegressor(n_estimators=100, random_state=42)
model_시간대 = RandomForestRegressor(n_estimators=100, random_state=42)

model_요일.fit(X_train_요일, y_train_요일)
model_시간대.fit(X_train_시간대, y_train_시간대)

# ------------------------
# 🎯 예측 (2024년 3분기 예측)
# ------------------------

# 검증 데이터 준비
X_test_요일 = test_data[list(요일_매출건수.values())]
X_test_시간대 = test_data[list(시간대_매출건수.values())]

# 요일별 매출 금액 예측
pred_요일 = model_요일.predict(X_test_요일)
pred_시간대 = model_시간대.predict(X_test_시간대)

# 요일별 매출 금액이 가장 적은 요일 (휴무 추천일)
휴무추천일 = 요일_columns[np.argmin(pred_요일)]

# 시간대별 매출 금액이 가장 적은 시간대 (휴식 추천 시간대)
휴식추천시간대 = 시간대_columns[np.argmin(pred_시간대)]

# ------------------------
# 🔥 혼잡 예상 시간대 예측 (요일 & 시간대 동시에 고려)
# ------------------------

# 모든 요일 & 시간대 조합 생성
요일_시간대_조합 = list(product(요일_columns, 시간대_columns))

# 조합별 예상 매출 건수 계산
혼잡_예상값 = []
for 요일, 시간대 in 요일_시간대_조합:
    요일_지표 = pred_요일[요일_columns.index(요일)]
    시간대_지표 = pred_시간대[시간대_columns.index(시간대)]
    혼잡_예상값.append((요일, 시간대, 요일_지표 + 시간대_지표))

# 상위 3개 혼잡 시간대 추출
혼잡_예상값 = sorted(혼잡_예상값, key=lambda x: x[2], reverse=True)[:3]
혼잡예상시간대 = [f"{요일}_{시간대}" for 요일, 시간대, _ in 혼잡_예상값]

# ------------------------
# 📢 결과 출력
# ------------------------
print("\n📌 2024년 3분기 예측 결과 📌")
print(f"✅ 휴무 추천일: {휴무추천일}")
print(f"✅ 혼잡 예상 시간대 (상위 3개): {', '.join(혼잡예상시간대)}")
print(f"✅ 휴식 추천 시간대: {휴식추천시간대}")

# ------------------------
# 📊 검증 결과 확인
# ------------------------

# 실제 검증 데이터와 비교
y_test_요일 = test_data[list(요일_매출금액.values())].mean(axis=1)
y_test_시간대 = test_data[list(시간대_매출금액.values())].mean(axis=1)

# MAE 계산 (평균 절대 오차)
from sklearn.metrics import mean_absolute_error

mae_요일 = mean_absolute_error(y_test_요일, pred_요일)
mae_시간대 = mean_absolute_error(y_test_시간대, pred_시간대)

print("\n📊 모델 검증 결과")
print(f"📉 요일별 매출 금액 예측 오차 (MAE): {mae_요일:.2f}")
print(f"📉 시간대별 매출 금액 예측 오차 (MAE): {mae_시간대:.2f}")

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
from itertools import product

# 데이터 불러오기
file_path = r"C:\Users\m\Desktop\머신러닝 사용 데이터\피처엔지니어링한통합데이터\피처엔지니어링일단다한통합데이터.csv"
data = pd.read_csv(file_path)

# ------------------------
# 🏢 사용자 입력 받는 함수
# ------------------------
def get_user_input():
    상권_코드_명 = input("상권 코드를 입력하세요: ")
    서비스_업종_코드_명 = input("업종명을 입력하세요: ")
    return 상권_코드_명, 서비스_업종_코드_명

# 사용자 입력 받기
상권_코드_명, 서비스_업종_코드_명 = get_user_input()

# ------------------------
# 📊 데이터 전처리
# ------------------------

# 1. 데이터 필터링 (사용자 입력한 상권 & 업종만 선택)
filtered_data = data[(data["상권_코드_명"] == 상권_코드_명) & (data["서비스_업종_코드_명"] == 서비스_업종_코드_명)]

if len(filtered_data) == 0:
    print("입력한 상권 코드 또는 업종명에 해당하는 데이터가 없습니다.")
    exit()

# 2. 기준_년분기_코드 기준 정렬 (시간순 정렬)
filtered_data = filtered_data.sort_values(by="기준_년분기_코드")

# 3. 2023년 4분기까지 학습, 2024년 데이터는 검증용
train_data = filtered_data[filtered_data["기준_년분기_코드"] <= 20234]
test_data = filtered_data[filtered_data["기준_년분기_코드"] > 20234]

if len(train_data) == 0 or len(test_data) == 0:
    print("학습 또는 검증 데이터가 충분하지 않습니다.")
    exit()

# 4. 요일 & 시간대 개별 피처로 학습을 위한 데이터 변환
요일_columns = ["월요일", "화요일", "수요일", "목요일", "금요일", "토요일", "일요일"]
시간대_columns = ["00~06", "06~11", "11~14", "14~17", "17~21", "21~24"]

# 요일별 매출 건수 / 매출 금액 데이터 변환
요일_매출건수 = {요일: f"{요일}_매출_건수" for 요일 in 요일_columns}
요일_매출금액 = {요일: f"{요일}_매출_금액" for 요일 in 요일_columns}

# 시간대별 매출 건수 / 매출 금액 데이터 변환
시간대_매출건수 = {시간대: f"시간대_{시간대}_매출_건수" for 시간대 in 시간대_columns}
시간대_매출금액 = {시간대: f"시간대_{시간대}_매출_금액" for 시간대 in 시간대_columns}

# ------------------------
# 🏆 머신러닝 모델 학습
# ------------------------

# 요일 학습을 위한 데이터셋 구성
X_train_요일 = train_data[list(요일_매출건수.values())]
y_train_요일 = train_data[list(요일_매출금액.values())].mean(axis=1)

# 시간대 학습을 위한 데이터셋 구성
X_train_시간대 = train_data[list(시간대_매출건수.values())]
y_train_시간대 = train_data[list(시간대_매출금액.values())].mean(axis=1)

# 모델 학습 (랜덤 포레스트 사용)
model_요일 = RandomForestRegressor(n_estimators=100, random_state=42)
model_시간대 = RandomForestRegressor(n_estimators=100, random_state=42)

model_요일.fit(X_train_요일, y_train_요일)
model_시간대.fit(X_train_시간대, y_train_시간대)

# ------------------------
# 🎯 예측 (2024년 3분기 예측)
# ------------------------

# 검증 데이터 준비
X_test_요일 = test_data[list(요일_매출건수.values())]
X_test_시간대 = test_data[list(시간대_매출건수.values())]

# 요일별 매출 금액 예측
pred_요일 = model_요일.predict(X_test_요일)
pred_시간대 = model_시간대.predict(X_test_시간대)

# 요일별 매출 금액이 가장 적은 요일 (휴무 추천일)
휴무추천일 = 요일_columns[np.argmin(pred_요일)]

# 시간대별 매출 금액이 가장 적은 시간대 (휴식 추천 시간대)
휴식추천시간대 = 시간대_columns[np.argmin(pred_시간대)]

# ------------------------
# 🔥 혼잡 예상 시간대 예측 (요일 & 시간대 동시에 고려)
# ------------------------

# 모든 요일 & 시간대 조합 생성
요일_시간대_조합 = list(product(요일_columns, 시간대_columns))

# 조합별 예상 매출 건수 계산
혼잡_예상값 = []
for 요일, 시간대 in 요일_시간대_조합:
    요일_지표 = pred_요일[요일_columns.index(요일)]
    시간대_지표 = pred_시간대[시간대_columns.index(시간대)]
    혼잡_예상값.append((요일, 시간대, 요일_지표 + 시간대_지표))

# 상위 3개 혼잡 시간대 추출
혼잡_예상값 = sorted(혼잡_예상값, key=lambda x: x[2], reverse=True)[:3]
혼잡예상시간대 = [f"{요일}_{시간대}" for 요일, 시간대, _ in 혼잡_예상값]

# ------------------------
# 📢 결과 출력
# ------------------------
print("\n📌 2024년 3분기 예측 결과 📌")
print(f"✅ 휴무 추천일: {휴무추천일}")
print(f"✅ 혼잡 예상 시간대 (상위 3개): {', '.join(혼잡예상시간대)}")
print(f"✅ 휴식 추천 시간대: {휴식추천시간대}")

# ------------------------
# 📊 검증 결과 확인
# ------------------------

# 실제 검증 데이터와 비교
y_test_요일 = test_data[list(요일_매출금액.values())].mean(axis=1)
y_test_시간대 = test_data[list(시간대_매출금액.values())].mean(axis=1)

# MAE 계산 (평균 절대 오차)
mae_요일 = mean_absolute_error(y_test_요일, pred_요일)
mae_시간대 = mean_absolute_error(y_test_시간대, pred_시간대)

print("\n📊 모델 검증 결과")
print(f"📉 요일별 매출 금액 예측 오차 (MAE): {mae_요일:.2f}")
print(f"📉 시간대별 매출 금액 예측 오차 (MAE): {mae_시간대:.2f}")

IndexError: index 2 is out of bounds for axis 0 with size 2

In [3]:
# ------------------------
# 🎯 예측 (2024년 3분기 예측)
# ------------------------

# 검증 데이터 준비
X_test_요일 = test_data[list(요일_매출건수.values())]
X_test_시간대 = test_data[list(시간대_매출건수.values())]

# 요일별 매출 금액 예측
pred_요일 = model_요일.predict(X_test_요일)
pred_시간대 = model_시간대.predict(X_test_시간대)

# pred_시간대의 크기 확인
print(f"pred_시간대 크기: {pred_시간대.shape}")  # 디버깅용 출력

# 요일별 매출 금액이 가장 적은 요일 (휴무 추천일)
휴무추천일 = 요일_columns[np.argmin(pred_요일)]

# 시간대별 매출 금액이 가장 적은 시간대 (휴식 추천 시간대)
휴식추천시간대 = 시간대_columns[np.argmin(pred_시간대)]

# ------------------------
# 🔥 혼잡 예상 시간대 예측 (요일 & 시간대 동시에 고려)
# ------------------------

# 모든 요일 & 시간대 조합 생성
요일_시간대_조합 = list(product(요일_columns, 시간대_columns))

# 조합별 예상 매출 건수 계산
혼잡_예상값 = []
for 요일, 시간대 in 요일_시간대_조합:
    요일_지표 = pred_요일[요일_columns.index(요일)]
    시간대_지표 = pred_시간대[시간대_columns.index(시간대)]
    혼잡_예상값.append((요일, 시간대, 요일_지표 + 시간대_지표))

# 상위 3개 혼잡 시간대 추출
혼잡_예상값 = sorted(혼잡_예상값, key=lambda x: x[2], reverse=True)[:3]
혼잡예상시간대 = [f"{요일}_{시간대}" for 요일, 시간대, _ in 혼잡_예상값]

# ------------------------
# 📢 결과 출력
# ------------------------
print("\n📌 2024년 3분기 예측 결과 📌")
print(f"✅ 휴무 추천일: {휴무추천일}")
print(f"✅ 혼잡 예상 시간대 (상위 3개): {', '.join(혼잡예상시간대)}")
print(f"✅ 휴식 추천 시간대: {휴식추천시간대}")

# ------------------------
# 📊 검증 결과 확인
# ------------------------

# 실제 검증 데이터와 비교
y_test_요일 = test_data[list(요일_매출금액.values())].mean(axis=1)
y_test_시간대 = test_data[list(시간대_매출금액.values())].mean(axis=1)

# MAE 계산 (평균 절대 오차)
mae_요일 = mean_absolute_error(y_test_요일, pred_요일)
mae_시간대 = mean_absolute_error(y_test_시간대, pred_시간대)

print("\n📊 모델 검증 결과")
print(f"📉 요일별 매출 금액 예측 오차 (MAE): {mae_요일:.2f}")
print(f"📉 시간대별 매출 금액 예측 오차 (MAE): {mae_시간대:.2f}")

pred_시간대 크기: (2,)


IndexError: index 2 is out of bounds for axis 0 with size 2

In [4]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor

def preprocess_data(df, 상권명, 업종명):
    df = df[(df['상권_코드_명'] == 상권명) & (df['서비스_업종_코드_명'] == 업종명)]
    df = df[df['기준_년분기_코드'] <= 20234]
    df_train = df[df['기준_년분기_코드'] < 20234]
    df_test = df[df['기준_년분기_코드'] == 20234]
    return df_train, df_test

def train_model(X, y):
    X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=42)
    model = RandomForestRegressor(n_estimators=100, random_state=42)
    model.fit(X_train, y_train)
    return model

def predict_sales(df_train, df_test):
    요일_columns = ['월요일_매출_금액', '화요일_매출_금액', '수요일_매출_금액', '목요일_매출_금액', '금요일_매출_금액', '토요일_매출_금액', '일요일_매출_금액']
    시간대_columns = ['시간대_00~06_매출_금액', '시간대_06~11_매출_금액', '시간대_11~14_매출_금액', '시간대_14~17_매출_금액', '시간대_17~21_매출_금액', '시간대_21~24_매출_금액']
    매출건수_columns = ['월요일_매출_건수', '화요일_매출_건수', '수요일_매출_건수', '목요일_매출_건수', '금요일_매출_건수', '토요일_매출_건수', '일요일_매출_건수']
    시간대_건수_columns = ['시간대_00~06_매출_건수', '시간대_06~11_매출_건수', '시간대_11~14_매출_건수', '시간대_14~17_매출_건수', '시간대_17~21_매출_건수', '시간대_21~24_매출_건수']
    
    # Map day names to numerical values
    요일_매핑 = {day: i for i, day in enumerate(요일_columns)}
    시간대_매핑 = {time: i for i, time in enumerate(시간대_columns)}
    
    # 휴무 추천일 예측 (요일별 매출 금액 기준)
    y_train = df_train[요일_columns].idxmin(axis=1).map(요일_매핑)
    X_train = df_train[요일_columns]
    요일_model = train_model(X_train, y_train)
    pred_요일 = 요일_model.predict(df_test[요일_columns])
    휴무추천일 = 요일_columns[np.argmin(pred_요일)]
    
    # 휴식 추천 시간대 예측 (시간대별 매출 금액 기준)
    y_train = df_train[시간대_columns].idxmin(axis=1).map(시간대_매핑)
    X_train = df_train[시간대_columns]
    시간대_model = train_model(X_train, y_train)
    pred_시간대 = 시간대_model.predict(df_test[시간대_columns])
    휴식추천시간대 = 시간대_columns[np.argmin(pred_시간대)]
    
    # 혼잡 예상 시간대 예측 (요일과 시간대 조합 - 매출 건수 기준)
    요일_시간대_조합 = [(요일, 시간대) for 요일 in 매출건수_columns for 시간대 in 시간대_건수_columns]
    혼잡_예상값 = []
    
    pred_요일 = df_test[매출건수_columns].mean(axis=0)
    pred_시간대 = df_test[시간대_건수_columns].mean(axis=0)
    
    for 요일, 시간대 in 요일_시간대_조합:
        요일_지표 = pred_요일[매출건수_columns.index(요일)]
        시간대_지표 = pred_시간대[시간대_건수_columns.index(시간대)]
        혼잡_예상값.append((요일, 시간대, 요일_지표 + 시간대_지표))
    
    # 상위 3개 혼잡 시간대 추출
    혼잡예상시간대 = sorted(혼잡_예상값, key=lambda x: x[2], reverse=True)[:3]
    혼잡예상시간대 = [f"{요일}_{시간대}" for 요일, 시간대, _ in 혼잡예상시간대]
    
    return 휴무추천일, 휴식추천시간대, 혼잡예상시간대

# 사용자 입력 및 실행
if __name__ == "__main__":
    df = pd.read_csv("C:\\Users\\m\\Desktop\\머신러닝 사용 데이터\\피처엔지니어링한통합데이터\\피처엔지니어링일단다한통합데이터.csv")
    
    상권명 = input("상권명을 입력하세요: ")
    업종명 = input("업종명을 입력하세요: ")
    df_train, df_test = preprocess_data(df, 상권명, 업종명)
    
    휴무추천일, 휴식추천시간대, 혼잡예상시간대 = predict_sales(df_train, df_test)
    
    print(f"휴무 추천일: {휴무추천일}")
    print(f"휴식 추천 시간대: {휴식추천시간대}")
    print(f"혼잡 예상 시간대 (상위 3개): {혼잡예상시간대}")

휴무 추천일: 월요일_매출_금액
휴식 추천 시간대: 시간대_00~06_매출_금액
혼잡 예상 시간대 (상위 3개): ['금요일_매출_건수_시간대_17~21_매출_건수', '월요일_매출_건수_시간대_17~21_매출_건수', '화요일_매출_건수_시간대_17~21_매출_건수']


C:\Users\m\AppData\Local\Temp\ipykernel_11076\178981583.py:51: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  요일_지표 = pred_요일[매출건수_columns.index(요일)]
C:\Users\m\AppData\Local\Temp\ipykernel_11076\178981583.py:52: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  시간대_지표 = pred_시간대[시간대_건수_columns.index(시간대)]


In [6]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor

def preprocess_data(df, 상권명, 업종명):
    df = df[(df['상권_코드_명'] == 상권명) & (df['서비스_업종_코드_명'] == 업종명)]
    df = df[df['기준_년분기_코드'] <= 20234]
    df_train = df[df['기준_년분기_코드'] < 20234]
    df_test = df[df['기준_년분기_코드'] == 20234]
    return df_train, df_test

def train_model(X, y):
    X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=42)
    model = RandomForestRegressor(n_estimators=100, random_state=42)
    model.fit(X_train, y_train)
    return model

def predict_sales(df_train, df_test):
    요일_columns = ['월요일_매출_금액', '화요일_매출_금액', '수요일_매출_금액', '목요일_매출_금액', '금요일_매출_금액', '토요일_매출_금액', '일요일_매출_금액']
    시간대_columns = ['시간대_00~06_매출_금액', '시간대_06~11_매출_금액', '시간대_11~14_매출_금액', '시간대_14~17_매출_금액', '시간대_17~21_매출_금액', '시간대_21~24_매출_금액']
    매출건수_columns = ['월요일_매출_건수', '화요일_매출_건수', '수요일_매출_건수', '목요일_매출_건수', '금요일_매출_건수', '토요일_매출_건수', '일요일_매출_건수']
    시간대_건수_columns = ['시간대_00~06_매출_건수', '시간대_06~11_매출_건수', '시간대_11~14_매출_건수', '시간대_14~17_매출_건수', '시간대_17~21_매출_건수', '시간대_21~24_매출_건수']
    
    # 요일 및 시간대 이름 변환용 리스트
    요일_이름 = ['월요일', '화요일', '수요일', '목요일', '금요일', '토요일', '일요일']
    시간대_이름 = ['00~06시', '06~11시', '11~14시', '14~17시', '17~21시', '21~24시']

    # 휴무 추천일 예측 (요일별 매출 금액 기준)
    y_train = df_train[요일_columns].idxmin(axis=1).map(lambda x: 요일_columns.index(x))
    X_train = df_train[요일_columns]
    요일_model = train_model(X_train, y_train)
    pred_요일 = 요일_model.predict(df_test[요일_columns])
    휴무추천일 = 요일_이름[int(round(np.mean(pred_요일)))]

    # 휴식 추천 시간대 예측 (시간대별 매출 금액 기준)
    y_train = df_train[시간대_columns].idxmin(axis=1).map(lambda x: 시간대_columns.index(x))
    X_train = df_train[시간대_columns]
    시간대_model = train_model(X_train, y_train)
    pred_시간대 = 시간대_model.predict(df_test[시간대_columns])
    휴식추천시간대 = 시간대_이름[int(round(np.mean(pred_시간대)))]

    # 혼잡 예상 시간대 예측 (요일과 시간대 조합 - 매출 건수 기준)
    요일_시간대_조합 = [(요일, 시간대) for 요일 in 매출건수_columns for 시간대 in 시간대_건수_columns]
    혼잡_예상값 = []

    pred_요일 = df_test[매출건수_columns].mean(axis=0)
    pred_시간대 = df_test[시간대_건수_columns].mean(axis=0)

    for 요일, 시간대 in 요일_시간대_조합:
        요일_지표 = pred_요일.iloc[매출건수_columns.index(요일)]
        시간대_지표 = pred_시간대.iloc[시간대_건수_columns.index(시간대)]

        혼잡_예상값.append((요일, 시간대, 요일_지표 + 시간대_지표))

    # 상위 3개 혼잡 시간대 추출
    혼잡예상시간대 = sorted(혼잡_예상값, key=lambda x: x[2], reverse=True)[:3]
    혼잡예상시간대 = [f"{요일_이름[매출건수_columns.index(요일)]} {시간대_이름[시간대_건수_columns.index(시간대)]}" for 요일, 시간대, _ in 혼잡예상시간대]

    return 휴무추천일, 휴식추천시간대, 혼잡예상시간대

# 사용자 입력 및 실행
if __name__ == "__main__":
    df = pd.read_csv("C:\\Users\\m\\Desktop\\머신러닝 사용 데이터\\피처엔지니어링한통합데이터\\피처엔지니어링일단다한통합데이터.csv")
    
    상권명 = input("상권명을 입력하세요: ")
    업종명 = input("업종명을 입력하세요: ")
    df_train, df_test = preprocess_data(df, 상권명, 업종명)
    
    휴무추천일, 휴식추천시간대, 혼잡예상시간대 = predict_sales(df_train, df_test)
    
    print(f"휴무 추천일: {휴무추천일}")
    print(f"휴식 추천 시간대: {휴식추천시간대}")
    print(f"혼잡 예상 시간대 (상위 3개): {혼잡예상시간대}")

휴무 추천일: 일요일
휴식 추천 시간대: 00~06시
혼잡 예상 시간대 (상위 3개): ['금요일 17~21시', '월요일 17~21시', '화요일 17~21시']


In [39]:
import pandas as pd

def predict_sales(df_train, df_test):
    요일_columns = ['월요일_매출_금액', '화요일_매출_금액', '수요일_매출_금액', '목요일_매출_금액', '금요일_매출_금액', '토요일_매출_금액', '일요일_매출_금액']
    매출건수_columns = ['월요일_매출_건수', '화요일_매출_건수', '수요일_매출_건수', '목요일_매출_건수', '금요일_매출_건수', '토요일_매출_건수', '일요일_매출_건수']
    시간대_건수_columns = ['시간대_00~06_매출_건수', '시간대_06~11_매출_건수', '시간대_11~14_매출_건수', '시간대_14~17_매출_건수', '시간대_17~21_매출_건수', '시간대_21~24_매출_건수']

    요일_이름 = ['월요일', '화요일', '수요일', '목요일', '금요일', '토요일', '일요일']
    시간대_이름 = ['00~06시', '06~11시', '11~14시', '14~17시', '17~21시', '21~24시']

    # 🟢 휴무 추천일 (매출 금액이 가장 낮은 요일 예측)
    pred_요일 = df_test[요일_columns].mean(axis=0)
    휴무추천일 = pred_요일.idxmin()
    휴무추천일 = 요일_이름[요일_columns.index(휴무추천일)]

    # 🔴 휴식 추천 시간대 (매출 건수가 가장 낮은 3개 시간대)
    pred_시간대 = df_test[시간대_건수_columns].mean(axis=0)
    휴식추천시간대 = pred_시간대.nsmallest(3).index.tolist()
    휴식추천시간대 = [시간대_이름[시간대_건수_columns.index(x)] for x in 휴식추천시간대]

    # 🔵 혼잡 예상 시간대 예측 (요일과 시간대 조합 - 매출 건수 기준)
    요일_시간대_조합 = [(요일, 시간대) for 요일 in 매출건수_columns for 시간대 in 시간대_건수_columns]
    혼잡_예상값 = []

    pred_요일 = df_test[매출건수_columns].mean(axis=0)
    pred_시간대 = df_test[시간대_건수_columns].mean(axis=0)

    for 요일, 시간대 in 요일_시간대_조합:
        요일_지표 = pred_요일.iloc[매출건수_columns.index(요일)]
        시간대_지표 = pred_시간대.iloc[시간대_건수_columns.index(시간대)]

        혼잡_예상값.append((요일, 시간대, 요일_지표 + 시간대_지표))

    # 상위 3개 혼잡 시간대 추출
    혼잡예상시간대 = sorted(혼잡_예상값, key=lambda x: x[2], reverse=True)[:3]
    혼잡예상시간대 = [f"{요일_이름[매출건수_columns.index(요일)]} {시간대_이름[시간대_건수_columns.index(시간대)]}" for 요일, 시간대, _ in 혼잡예상시간대]

    return 휴무추천일, 휴식추천시간대, 혼잡예상시간대


# 사용자 입력 및 실행
if __name__ == "__main__":
    df = pd.read_csv("C:\\Users\\m\\Desktop\\머신러닝 사용 데이터\\피처엔지니어링한통합데이터\\피처엔지니어링일단다한통합데이터.csv")
    
    상권명 = input("상권명을 입력하세요: ")
    업종명 = input("업종명을 입력하세요: ")
    df_train, df_test = preprocess_data(df, 상권명, 업종명)
    
    휴무추천일, 휴식추천시간대, 혼잡예상시간대 = predict_sales(df_train, df_test)
    
    print(f"휴무 추천일: {휴무추천일}")
    print(f"휴식 추천 시간대 (상위 3개): {휴식추천시간대}")
    print(f"혼잡 예상 시간대 (상위 3개): {혼잡예상시간대}")

휴무 추천일: 일요일
휴식 추천 시간대 (상위 3개): ['00~06시', '06~11시', '21~24시']
혼잡 예상 시간대 (상위 3개): ['금요일 17~21시', '월요일 17~21시', '화요일 17~21시']


In [41]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor

def preprocess_data(df, 상권명, 업종명):
    df = df[(df['상권_코드_명'] == 상권명) & (df['서비스_업종_코드_명'] == 업종명)]
    df = df[df['기준_년분기_코드'] <= 20234]
    df_train = df[df['기준_년분기_코드'] < 20234]
    df_test = df[df['기준_년분기_코드'] == 20234]
    return df_train, df_test

def train_model(X, y):
    X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=42)
    model = RandomForestRegressor(n_estimators=100, random_state=42)
    model.fit(X_train, y_train)
    return model

def predict_sales(df_train, df_test):
    요일_columns = ['월요일_매출_금액', '화요일_매출_금액', '수요일_매출_금액', '목요일_매출_금액', '금요일_매출_금액', '토요일_매출_금액', '일요일_매출_금액']
    매출건수_columns = ['월요일_매출_건수', '화요일_매출_건수', '수요일_매출_건수', '목요일_매출_건수', '금요일_매출_건수', '토요일_매출_건수', '일요일_매출_건수']
    시간대_건수_columns = ['시간대_00~06_매출_건수', '시간대_06~11_매출_건수', '시간대_11~14_매출_건수', '시간대_14~17_매출_건수', '시간대_17~21_매출_건수', '시간대_21~24_매출_건수']
    
    요일_이름 = ['월요일', '화요일', '수요일', '목요일', '금요일', '토요일', '일요일']
    시간대_이름 = ['00~06시', '06~11시', '11~14시', '14~17시', '17~21시', '21~24시']

    # 🟢 휴무 추천일 예측 (요일별 매출 금액 기준)
    요일_model = train_model(df_train[요일_columns], df_train[요일_columns].idxmin(axis=1).map(lambda x: 요일_columns.index(x)))
    pred_요일 = 요일_model.predict(df_test[요일_columns])
    휴무추천일 = 요일_이름[int(round(np.mean(pred_요일)))]
    
    # 🔴 휴식 추천 시간대 예측 (시간대별 매출 건수 기준, 최저 3개 선정)
    시간대_model = train_model(df_train[시간대_건수_columns], df_train[시간대_건수_columns].idxmin(axis=1).map(lambda x: 시간대_건수_columns.index(x)))
    pred_시간대 = 시간대_model.predict(df_test[시간대_건수_columns])
    휴식추천시간대 = [시간대_이름[idx] for idx in np.argsort(pred_시간대)[:3]]
    
    # 🔵 혼잡 예상 시간대 예측 (요일 & 시간대 조합 - 매출 건수 기준)
    혼잡_예상값 = []
    요일_model = train_model(df_train[매출건수_columns], df_train[매출건수_columns].mean(axis=1))
    시간대_model = train_model(df_train[시간대_건수_columns], df_train[시간대_건수_columns].mean(axis=1))
    pred_요일 = 요일_model.predict(df_test[매출건수_columns])
    pred_시간대 = 시간대_model.predict(df_test[시간대_건수_columns])

    for i, 요일 in enumerate(매출건수_columns):
        for j, 시간대 in enumerate(시간대_건수_columns):
            혼잡_예상값.append((i, j, pred_요일[i] + pred_시간대[j]))
    
    # 상위 3개 혼잡 시간대 추출
    혼잡예상시간대 = sorted(혼잡_예상값, key=lambda x: x[2], reverse=True)[:3]
    혼잡예상시간대 = [f"{요일_이름[i]} {시간대_이름[j]}" for i, j, _ in 혼잡예상시간대]

    return 휴무추천일, 휴식추천시간대, 혼잡예상시간대

# 사용자 입력 및 실행
if __name__ == "__main__":
    df = pd.read_csv("C:\\Users\\m\\Desktop\\머신러닝 사용 데이터\\피처엔지니어링한통합데이터\\피처엔지니어링일단다한통합데이터.csv")
    
    상권명 = input("상권명을 입력하세요: ")
    업종명 = input("업종명을 입력하세요: ")
    df_train, df_test = preprocess_data(df, 상권명, 업종명)
    
    휴무추천일, 휴식추천시간대, 혼잡예상시간대 = predict_sales(df_train, df_test)
    
    print(f"휴무 추천일: {휴무추천일}")
    print(f"휴식 추천 시간대 (상위 3개): {휴식추천시간대}")
    print(f"혼잡 예상 시간대 (상위 3개): {혼잡예상시간대}")

IndexError: index 1 is out of bounds for axis 0 with size 1

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor

def preprocess_data(df, 상권명, 업종명):
    df = df[(df['상권_코드_명'] == 상권명) & (df['서비스_업종_코드_명'] == 업종명)]
    df = df[df['기준_년분기_코드'] <= 20234]
    df_train = df[df['기준_년분기_코드'] < 20234]
    df_test = df[df['기준_년분기_코드'] == 20234]
    return df_train, df_test

def train_model(X, y):
    X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=42)
    model = RandomForestRegressor(n_estimators=100, random_state=42)
    model.fit(X_train, y_train)
    return model

def predict_sales(df_train, df_test):
    요일_columns = ['월요일_매출_금액', '화요일_매출_금액', '수요일_매출_금액', '목요일_매출_금액', '금요일_매출_금액', '토요일_매출_금액', '일요일_매출_금액']
    매출건수_columns = ['월요일_매출_건수', '화요일_매출_건수', '수요일_매출_건수', '목요일_매출_건수', '금요일_매출_건수', '토요일_매출_건수', '일요일_매출_건수']
    시간대_건수_columns = ['시간대_00~06_매출_건수', '시간대_06~11_매출_건수', '시간대_11~14_매출_건수', '시간대_14~17_매출_건수', '시간대_17~21_매출_건수', '시간대_21~24_매출_건수']
    
    요일_이름 = ['월요일', '화요일', '수요일', '목요일', '금요일', '토요일', '일요일']
    시간대_이름 = ['00~06시', '06~11시', '11~14시', '14~17시', '17~21시', '21~24시']

    # 🟢 휴무 추천일 예측
    요일_model = train_model(df_train[요일_columns], df_train[요일_columns].idxmin(axis=1).map(lambda x: 요일_columns.index(x)))
    pred_요일 = 요일_model.predict(df_test[요일_columns])
    
    # 예측값이 여러 개일 경우 평균을 사용
    pred_요일 = np.mean(pred_요일)  
    휴무추천일 = 요일_이름[int(round(pred_요일))]

    # 🔴 휴식 추천 시간대 예측
    시간대_model = train_model(df_train[시간대_건수_columns], df_train[시간대_건수_columns].idxmin(axis=1).map(lambda x: 시간대_건수_columns.index(x)))
    pred_시간대 = 시간대_model.predict(df_test[시간대_건수_columns])
    
    # 예측값이 여러 개일 경우 평균을 사용
    pred_시간대 = np.mean(pred_시간대, axis=0)  
    휴식추천시간대 = [시간대_이름[idx] for idx in np.argsort(pred_시간대)[:3]]
    
    # 🔵 혼잡 예상 시간대 예측
    혼잡_예상값 = []

    # 요일과 시간대별 평균을 예측
    요일_model = train_model(df_train[매출건수_columns], df_train[매출건수_columns].mean(axis=1))
    시간대_model = train_model(df_train[시간대_건수_columns], df_train[시간대_건수_columns].mean(axis=1))
    
    pred_요일 = np.mean(요일_model.predict(df_test[매출건수_columns]), axis=0)  
    pred_시간대 = np.mean(시간대_model.predict(df_test[시간대_건수_columns]), axis=0)

    for i in range(len(매출건수_columns)):
        for j in range(len(시간대_건수_columns)):
            혼잡_예상값.append((i, j, pred_요일 + pred_시간대))

    # 상위 3개 혼잡 시간대 추출
    혼잡예상시간대 = sorted(혼잡_예상값, key=lambda x: x[2], reverse=True)[:3]
    혼잡예상시간대 = [f"{요일_이름[i]} {시간대_이름[j]}" for i, j, _ in 혼잡예상시간대]

    return 휴무추천일, 휴식추천시간대, 혼잡예상시간대

# 사용자 입력 및 실행
if __name__ == "__main__":
    df = pd.read_csv("C:\\Users\\m\\Desktop\\머신러닝 사용 데이터\\피처엔지니어링한통합데이터\\피처엔지니어링일단다한통합데이터.csv")
    
    상권명 = input("상권명을 입력하세요: ")
    업종명 = input("업종명을 입력하세요: ")
    df_train, df_test = preprocess_data(df, 상권명, 업종명)
    
    휴무추천일, 휴식추천시간대, 혼잡예상시간대 = predict_sales(df_train, df_test)
    
    print(f"휴무 추천일: {휴무추천일}")
    print(f"휴식 추천 시간대 (상위 3개): {휴식추천시간대}")
    print(f"혼잡 예상 시간대 (상위 3개): {혼잡예상시간대}")

휴무 추천일: 일요일
휴식 추천 시간대 (상위 3개): ['00~06시']
혼잡 예상 시간대 (상위 3개): ['월요일 00~06시', '월요일 06~11시', '월요일 11~14시']


In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor

def preprocess_data(df, 상권명, 업종명):
    df = df[(df['상권_코드_명'] == 상권명) & (df['서비스_업종_코드_명'] == 업종명)]
    df = df[df['기준_년분기_코드'] <= 20234]
    df_train = df[df['기준_년분기_코드'] < 20234]
    df_test = df[df['기준_년분기_코드'] == 20234]
    return df_train, df_test

def train_model(X, y):
    X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=42)
    model = RandomForestRegressor(n_estimators=100, random_state=42)
    model.fit(X_train, y_train)
    return model

def predict_sales(df_train, df_test):
    요일_columns = ['월요일_매출_금액', '화요일_매출_금액', '수요일_매출_금액', '목요일_매출_금액', '금요일_매출_금액', '토요일_매출_금액', '일요일_매출_금액']
    매출건수_columns = ['월요일_매출_건수', '화요일_매출_건수', '수요일_매출_건수', '목요일_매출_건수', '금요일_매출_건수', '토요일_매출_건수', '일요일_매출_건수']
    시간대_건수_columns = ['시간대_00~06_매출_건수', '시간대_06~11_매출_건수', '시간대_11~14_매출_건수', '시간대_14~17_매출_건수', '시간대_17~21_매출_건수', '시간대_21~24_매출_건수']
    
    요일_이름 = ['월요일', '화요일', '수요일', '목요일', '금요일', '토요일', '일요일']
    시간대_이름 = ['00~06시', '06~11시', '11~14시', '14~17시', '17~21시', '21~24시']

    # 🟢 휴무 추천일 예측
    요일_model = train_model(df_train[요일_columns], df_train[요일_columns].idxmin(axis=1).map(lambda x: 요일_columns.index(x)))
    pred_요일 = np.mean(요일_model.predict(df_test[요일_columns]))  
    휴무추천일 = 요일_이름[int(round(pred_요일))]

    # 🔴 휴식 추천 시간대 예측 (각 시간대별로 학습 후, 개별 예측값을 정렬)
    시간대_pred = []
    for col in 시간대_건수_columns:
        model = train_model(df_train.drop(columns=[col]), df_train[col])
        pred = model.predict(df_test.drop(columns=[col])).mean()
        시간대_pred.append(pred)

    # 가장 낮은 3개 시간대 선택
    휴식추천시간대 = [시간대_이름[idx] for idx in np.argsort(시간대_pred)[:3]]

    # 🔵 혼잡 예상 시간대 예측
    혼잡_예상값 = []

    요일_model = train_model(df_train[매출건수_columns], df_train[매출건수_columns].mean(axis=1))
    시간대_model = train_model(df_train[시간대_건수_columns], df_train[시간대_건수_columns].mean(axis=1))
    
    pred_요일 = np.mean(요일_model.predict(df_test[매출건수_columns]), axis=0)  
    pred_시간대 = np.mean(시간대_model.predict(df_test[시간대_건수_columns]), axis=0)

    for i in range(len(매출건수_columns)):
        for j in range(len(시간대_건수_columns)):
            혼잡_예상값.append((i, j, pred_요일 + pred_시간대))

    # 상위 3개 혼잡 시간대 추출
    혼잡예상시간대 = sorted(혼잡_예상값, key=lambda x: x[2], reverse=True)[:3]
    혼잡예상시간대 = [f"{요일_이름[i]} {시간대_이름[j]}" for i, j, _ in 혼잡예상시간대]

    return 휴무추천일, 휴식추천시간대, 혼잡예상시간대

# 사용자 입력 및 실행
if __name__ == "__main__":
    df = pd.read_csv("C:\\Users\\m\\Desktop\\머신러닝 사용 데이터\\피처엔지니어링한통합데이터\\피처엔지니어링일단다한통합데이터.csv")
    
    상권명 = input("상권명을 입력하세요: ")
    업종명 = input("업종명을 입력하세요: ")
    df_train, df_test = preprocess_data(df, 상권명, 업종명)
    
    휴무추천일, 휴식추천시간대, 혼잡예상시간대 = predict_sales(df_train, df_test)
    
    print(f"휴무 추천일: {휴무추천일}")
    print(f"휴식 추천 시간대 (상위 3개): {휴식추천시간대}")
    print(f"혼잡 예상 시간대 (상위 3개): {혼잡예상시간대}")

ValueError: could not convert string to float: 'D'

최종코드드

In [4]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor

def preprocess_data(df, 상권명, 업종명):
    df = df[(df['상권_코드_명'] == 상권명) & (df['서비스_업종_코드_명'] == 업종명)]
    df = df[df['기준_년분기_코드'] <= 20234]
    df_train = df[df['기준_년분기_코드'] < 20234]
    df_test = df[df['기준_년분기_코드'] == 20234]
    
    # 🚨 수치형 데이터만 선택
    df_train = df_train.select_dtypes(include=[np.number])
    df_test = df_test.select_dtypes(include=[np.number])
    
    return df_train, df_test

def train_model(X, y):
    X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=42)
    model = RandomForestRegressor(n_estimators=100, random_state=42)
    model.fit(X_train, y_train)
    return model

def predict_sales(df_train, df_test):
    요일_columns = ['월요일_매출_금액', '화요일_매출_금액', '수요일_매출_금액', '목요일_매출_금액', '금요일_매출_금액', '토요일_매출_금액', '일요일_매출_금액']
    매출건수_columns = ['월요일_매출_건수', '화요일_매출_건수', '수요일_매출_건수', '목요일_매출_건수', '금요일_매출_건수', '토요일_매출_건수', '일요일_매출_건수']
    시간대_건수_columns = ['시간대_00~06_매출_건수', '시간대_06~11_매출_건수', '시간대_11~14_매출_건수', '시간대_14~17_매출_건수', '시간대_17~21_매출_건수', '시간대_21~24_매출_건수']
    
    요일_이름 = ['월요일', '화요일', '수요일', '목요일', '금요일', '토요일', '일요일']
    시간대_이름 = ['00~06시', '06~11시', '11~14시', '14~17시', '17~21시', '21~24시']

    # 🟢 휴무 추천일 예측
    요일_model = train_model(df_train[요일_columns], df_train[요일_columns].idxmin(axis=1).map(lambda x: 요일_columns.index(x)))
    pred_요일 = np.mean(요일_model.predict(df_test[요일_columns]))  
    휴무추천일 = 요일_이름[int(round(pred_요일))]

    # 🔴 휴식 추천 시간대 예측
    시간대_pred = []
    for col in 시간대_건수_columns:
        X_train = df_train.drop(columns=시간대_건수_columns)  
        y_train = df_train[col]  
        if X_train.empty:
            continue  

        model = train_model(X_train, y_train)
        X_test = df_test.drop(columns=시간대_건수_columns)
        if X_test.empty:
            continue  

        pred = model.predict(X_test).mean()
        시간대_pred.append((col, pred))

    시간대_pred = sorted(시간대_pred, key=lambda x: x[1])[:3]
    휴식추천시간대 = [시간대_이름[시간대_건수_columns.index(col)] for col, _ in 시간대_pred]

    # 🔵 혼잡 예상 시간대 예측 (각 요일 & 시간대 개별 예측)
    요일_pred_list = []
    시간대_pred_list = []

    for col in 매출건수_columns:
        model = train_model(df_train.drop(columns=매출건수_columns), df_train[col])
        pred = model.predict(df_test.drop(columns=매출건수_columns)).mean()
        요일_pred_list.append(pred)

    for col in 시간대_건수_columns:
        model = train_model(df_train.drop(columns=시간대_건수_columns), df_train[col])
        pred = model.predict(df_test.drop(columns=시간대_건수_columns)).mean()
        시간대_pred_list.append(pred)

    # 🔹 요일별, 시간대별 예측값을 조합하여 혼잡 예상 시간대 선정
    혼잡_예상값 = []
    for i, 요일_pred in enumerate(요일_pred_list):
        for j, 시간대_pred in enumerate(시간대_pred_list):
            혼잡_예상값.append((i, j, 요일_pred + 시간대_pred))  # 🔹 개별 예측값 합산

    혼잡예상시간대 = sorted(혼잡_예상값, key=lambda x: x[2], reverse=True)[:3]
    혼잡예상시간대 = [f"{요일_이름[i]} {시간대_이름[j]}" for i, j, _ in 혼잡예상시간대]

    return 휴무추천일, 휴식추천시간대, 혼잡예상시간대

# 사용자 입력 및 실행
if __name__ == "__main__":
    df = pd.read_csv("C:\\Users\\m\\Desktop\\머신러닝 사용 데이터\\피처엔지니어링한통합데이터\\피처엔지니어링일단다한통합데이터.csv")
    
    상권명 = input("상권명을 입력하세요: ")
    업종명 = input("업종명을 입력하세요: ")
    df_train, df_test = preprocess_data(df, 상권명, 업종명)
    
    휴무추천일, 휴식추천시간대, 혼잡예상시간대 = predict_sales(df_train, df_test)
    
    print(f"휴무 추천일: {휴무추천일}")
    print(f"휴식 추천 시간대 (상위 3개): {휴식추천시간대}")
    print(f"혼잡 예상 시간대 (상위 3개): {혼잡예상시간대}")

휴무 추천일: 일요일
휴식 추천 시간대 (상위 3개): ['00~06시', '06~11시', '21~24시']
혼잡 예상 시간대 (상위 3개): ['월요일 17~21시', '금요일 17~21시', '화요일 17~21시']


In [8]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

def train_model(X, y):
    X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=42)
    model = RandomForestRegressor(n_estimators=100, random_state=42)
    model.fit(X_train, y_train)
    
    # 검증 데이터에 대한 예측값 계산
    y_valid_pred = model.predict(X_valid)
    mae = mean_absolute_error(y_valid, y_valid_pred)  # MAE 계산
    
    return model, mae  # 모델과 검증 오차 반환

def predict_sales(df_train, df_test):
    요일_columns = ['월요일_매출_금액', '화요일_매출_금액', '수요일_매출_금액', '목요일_매출_금액', '금요일_매출_금액', '토요일_매출_금액', '일요일_매출_금액']
    매출건수_columns = ['월요일_매출_건수', '화요일_매출_건수', '수요일_매출_건수', '목요일_매출_건수', '금요일_매출_건수', '토요일_매출_건수', '일요일_매출_건수']
    시간대_건수_columns = ['시간대_00~06_매출_건수', '시간대_06~11_매출_건수', '시간대_11~14_매출_건수', '시간대_14~17_매출_건수', '시간대_17~21_매출_건수', '시간대_21~24_매출_건수']
    
    요일_이름 = ['월요일', '화요일', '수요일', '목요일', '금요일', '토요일', '일요일']
    시간대_이름 = ['00~06시', '06~11시', '11~14시', '14~17시', '17~21시', '21~24시']

    # 🟢 휴무 추천일 예측
    요일_model, 요일_mae = train_model(df_train[요일_columns], df_train[요일_columns].idxmin(axis=1).map(lambda x: 요일_columns.index(x)))
    pred_요일 = np.mean(요일_model.predict(df_test[요일_columns]))  
    휴무추천일 = 요일_이름[int(round(pred_요일))]

    # 🔴 휴식 추천 시간대 예측
    시간대_pred = []
    시간대_mae_list = []  # 검증 오차 저장
    for col in 시간대_건수_columns:
        X_train = df_train.drop(columns=시간대_건수_columns)  
        y_train = df_train[col]  
        if X_train.empty:
            continue  

        model, mae = train_model(X_train, y_train)
        X_test = df_test.drop(columns=시간대_건수_columns)
        if X_test.empty:
            continue  

        pred = model.predict(X_test).mean()
        시간대_pred.append((col, pred))
        시간대_mae_list.append(mae)

    시간대_pred = sorted(시간대_pred, key=lambda x: x[1])[:3]
    휴식추천시간대 = [시간대_이름[시간대_건수_columns.index(col)] for col, _ in 시간대_pred]
    평균_시간대_mae = np.mean(시간대_mae_list)  # 평균 MAE 계산

    # 🔵 혼잡 예상 시간대 예측
    요일_pred_list = []
    시간대_pred_list = []
    요일_mae_list = []
    시간대_mae_list = []

    for col in 매출건수_columns:
        model, mae = train_model(df_train.drop(columns=매출건수_columns), df_train[col])
        pred = model.predict(df_test.drop(columns=매출건수_columns)).mean()
        요일_pred_list.append(pred)
        요일_mae_list.append(mae)

    for col in 시간대_건수_columns:
        model, mae = train_model(df_train.drop(columns=시간대_건수_columns), df_train[col])
        pred = model.predict(df_test.drop(columns=시간대_건수_columns)).mean()
        시간대_pred_list.append(pred)
        시간대_mae_list.append(mae)

    # 🔹 요일 & 시간대 조합
    혼잡_예상값 = []
    for i, 요일_pred in enumerate(요일_pred_list):
        for j, 시간대_pred in enumerate(시간대_pred_list):
            혼잡_예상값.append((i, j, 요일_pred + 시간대_pred))

    혼잡예상시간대 = sorted(혼잡_예상값, key=lambda x: x[2], reverse=True)[:3]
    혼잡예상시간대 = [f"{요일_이름[i]} {시간대_이름[j]}" for i, j, _ in 혼잡예상시간대]
    평균_혼잡_mae = (np.mean(요일_mae_list) + np.mean(시간대_mae_list)) / 2

    # 🔽 검증 데이터 평가 결과 출력
    print(f"📌 검증 데이터 평가 결과")
    print(f" - 휴무 추천일 MAE: {요일_mae:.3f}")
    print(f" - 휴식 추천 시간대 MAE: {평균_시간대_mae:.3f}")
    print(f" - 혼잡 예상 시간대 MAE: {평균_혼잡_mae:.3f}")

    return 휴무추천일, 휴식추천시간대, 혼잡예상시간대

# 사용자 입력 및 실행
if __name__ == "__main__":
    df = pd.read_csv("C:\\Users\\m\\Desktop\\머신러닝 사용 데이터\\피처엔지니어링한통합데이터\\피처엔지니어링일단다한통합데이터.csv")
    
    상권명 = input("상권명을 입력하세요: ")
    업종명 = input("업종명을 입력하세요: ")
    df_train, df_test = preprocess_data(df, 상권명, 업종명)
    
    휴무추천일, 휴식추천시간대, 혼잡예상시간대 = predict_sales(df_train, df_test)
    
    print(f"휴무 추천일: {휴무추천일}")
    print(f"휴식 추천 시간대 (상위 3개): {휴식추천시간대}")
    print(f"혼잡 예상 시간대 (상위 3개): {혼잡예상시간대}")

📌 검증 데이터 평가 결과
 - 휴무 추천일 MAE: 0.265
 - 휴식 추천 시간대 MAE: 4.840
 - 혼잡 예상 시간대 MAE: 5.054
휴무 추천일: 일요일
휴식 추천 시간대 (상위 3개): ['00~06시', '06~11시', '11~14시']
혼잡 예상 시간대 (상위 3개): ['화요일 17~21시', '금요일 17~21시', '수요일 17~21시']


In [18]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

def train_model(X, y):
    X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=42)
    model = RandomForestRegressor(n_estimators=100, random_state=42)
    model.fit(X_train, y_train)
    
    y_valid_pred = model.predict(X_valid)
    mae = mean_absolute_error(y_valid, y_valid_pred)
    mse = mean_squared_error(y_valid, y_valid_pred)
    r2 = r2_score(y_valid, y_valid_pred)
    
    return model, mae, mse, r2

def predict_sales(df_train, df_test):
    요일_columns = ['월요일_매출_금액', '화요일_매출_금액', '수요일_매출_금액', '목요일_매출_금액', '금요일_매출_금액', '토요일_매출_금액', '일요일_매출_금액']
    매출건수_columns = ['월요일_매출_건수', '화요일_매출_건수', '수요일_매출_건수', '목요일_매출_건수', '금요일_매출_건수', '토요일_매출_건수', '일요일_매출_건수']
    시간대_건수_columns = ['시간대_00~06_매출_건수', '시간대_06~11_매출_건수', '시간대_11~14_매출_건수', '시간대_14~17_매출_건수', '시간대_17~21_매출_건수', '시간대_21~24_매출_건수']
    
    요일_이름 = ['월요일', '화요일', '수요일', '목요일', '금요일', '토요일', '일요일']
    시간대_이름 = ['00~06시', '06~11시', '11~14시', '14~17시', '17~21시', '21~24시']

    # 🟢 휴무 추천일 예측
    요일_model, 요일_mae, 요일_mse, 요일_r2 = train_model(df_train[요일_columns], df_train[요일_columns].idxmin(axis=1).map(lambda x: 요일_columns.index(x)))
    pred_요일 = np.mean(요일_model.predict(df_test[요일_columns]))  
    휴무추천일 = 요일_이름[int(round(pred_요일))]

    # 🔴 휴식 추천 시간대 예측
    시간대_pred = []
    시간대_mae_list, 시간대_mse_list, 시간대_r2_list = [], [], []
    for col in 시간대_건수_columns:
        X_train = df_train.drop(columns=시간대_건수_columns)
        y_train = df_train[col]
        if X_train.empty:
            continue

        model, mae, mse, r2 = train_model(X_train, y_train)
        X_test = df_test.drop(columns=시간대_건수_columns)
        if X_test.empty:
            continue

        pred = model.predict(X_test).mean()
        시간대_pred.append((col, pred))
        시간대_mae_list.append(mae)
        시간대_mse_list.append(mse)
        시간대_r2_list.append(r2)

    시간대_pred = sorted(시간대_pred, key=lambda x: x[1])[:3]
    휴식추천시간대 = [시간대_이름[시간대_건수_columns.index(col)] for col, _ in 시간대_pred]

    # 🔵 혼잡 예상 시간대 예측
    요일_pred_list = []
    시간대_pred_list = []
    요일_mae_list, 시간대_mae_list = [], []
    요일_mse_list, 시간대_mse_list = [], []
    요일_r2_list, 시간대_r2_list = [], []

    for col in 매출건수_columns:
        model, mae, mse, r2 = train_model(df_train.drop(columns=매출건수_columns), df_train[col])
        pred = model.predict(df_test.drop(columns=매출건수_columns)).mean()
        요일_pred_list.append(pred)
        요일_mae_list.append(mae)
        요일_mse_list.append(mse)
        요일_r2_list.append(r2)

    for col in 시간대_건수_columns:
        model, mae, mse, r2 = train_model(df_train.drop(columns=시간대_건수_columns), df_train[col])
        pred = model.predict(df_test.drop(columns=시간대_건수_columns)).mean()
        시간대_pred_list.append(pred)
        시간대_mae_list.append(mae)
        시간대_mse_list.append(mse)
        시간대_r2_list.append(r2)

    혼잡_예상값 = []
    for i, 요일_pred in enumerate(요일_pred_list):
        for j, 시간대_pred in enumerate(시간대_pred_list):
            혼잡_예상값.append((i, j, 요일_pred + 시간대_pred))

    혼잡예상시간대 = sorted(혼잡_예상값, key=lambda x: x[2], reverse=True)[:3]
    혼잡예상시간대 = [f"{요일_이름[i]} {시간대_이름[j]}" for i, j, _ in 혼잡예상시간대]

    # 평균 정확도 지표
    평균_mae = np.mean(요일_mae_list + 시간대_mae_list)
    평균_mse = np.mean(요일_mse_list + 시간대_mse_list)
    평균_r2 = np.mean(요일_r2_list + 시간대_r2_list)

    print("📌 검증 데이터 평가 결과 요약")
    print(f" - 평균 MAE: {평균_mae:.3f}")
    print(f" - 평균 MSE: {평균_mse:.3f}")
    print(f" - 평균 R²: {평균_r2:.3f}")

    return 휴무추천일, 휴식추천시간대, 혼잡예상시간대

if __name__ == "__main__":
    df = pd.read_csv("C:\\Users\\m\\Desktop\\머신러닝 사용 데이터\\피처엔지니어링한통합데이터\\피처엔지니어링일단다한통합데이터.csv")

    상권명 = input("상권명을 입력하세요: ")
    업종명 = input("업종명을 입력하세요: ")
    df_train, df_test = preprocess_data(df, 상권명, 업종명)

    휴무추천일, 휴식추천시간대, 혼잡예상시간대 = predict_sales(df_train, df_test)

    print(f"휴무 추천일: {휴무추천일}")
    print(f"휴식 추천 시간대 (상위 3개): {휴식추천시간대}")
    print(f"혼잡 예상 시간대 (상위 3개): {혼잡예상시간대}")

📌 검증 데이터 평가 결과 요약
 - 평균 MAE: 435.003
 - 평균 MSE: 504813.077
 - 평균 R²: 0.198
휴무 추천일: 일요일
휴식 추천 시간대 (상위 3개): ['00~06시', '21~24시', '17~21시']
혼잡 예상 시간대 (상위 3개): ['월요일 14~17시', '수요일 14~17시', '화요일 14~17시']


새로운 변수를 추가한 코드

In [8]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

def train_model(X, y):
    X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=42)
    model = RandomForestRegressor(n_estimators=100, random_state=42)
    model.fit(X_train, y_train)
    
    y_valid_pred = model.predict(X_valid)
    mae = mean_absolute_error(y_valid, y_valid_pred)
    mse = mean_squared_error(y_valid, y_valid_pred)
    r2 = r2_score(y_valid, y_valid_pred)
    
    return model, mae, mse, r2

def preprocess_data(df, 상권명, 업종명):
    df_filtered = df[(df['상권코드명'] == 상권명) & (df['업종명'] == 업종명)]
    df_train = df_filtered[df_filtered['년분기'] <= 20234]
    df_test = df_filtered[df_filtered['년분기'] == 20241]
    return df_train, df_test

def predict_sales(df_train, df_test):
    요일_columns = ['월요일_매출_금액', '화요일_매출_금액', '수요일_매출_금액', '목요일_매출_금액', '금요일_매출_금액', '토요일_매출_금액', '일요일_매출_금액']
    매출건수_columns = ['월요일_매출_건수', '화요일_매출_건수', '수요일_매출_건수', '목요일_매출_건수', '금요일_매출_건수', '토요일_매출_건수', '일요일_매출_건수']
    시간대_건수_columns = ['시간대_00~06_매출_건수', '시간대_06~11_매출_건수', '시간대_11~14_매출_건수', '시간대_14~17_매출_건수', '시간대_17~21_매출_건수', '시간대_21~24_매출_건수']
    
    요일_이름 = ['월요일', '화요일', '수요일', '목요일', '금요일', '토요일', '일요일']
    시간대_이름 = ['00~06시', '06~11시', '11~14시', '14~17시', '17~21시', '21~24시']
    
    df_train['요일별_매출_집중도'] = df_train[요일_columns].div(df_train[요일_columns].sum(axis=1), axis=0)
    df_train['시간대별_매출_집중도'] = df_train[시간대_건수_columns].div(df_train[시간대_건수_columns].sum(axis=1), axis=0)
    df_train['영업_효율성_지수'] = df_train['총매출_금액'] / df_train['총영업시간']
    
    df_test['요일별_매출_집중도'] = df_test[요일_columns].div(df_test[요일_columns].sum(axis=1), axis=0)
    df_test['시간대별_매출_집중도'] = df_test[시간대_건수_columns].div(df_test[시간대_건수_columns].sum(axis=1), axis=0)
    df_test['영업_효율성_지수'] = df_test['총매출_금액'] / df_test['총영업시간']
    
    요일_model, _, _, _ = train_model(df_train[요일_columns], df_train[요일_columns].idxmin(axis=1).map(lambda x: 요일_columns.index(x)))
    pred_요일 = np.mean(요일_model.predict(df_test[요일_columns]))  
    휴무추천일 = 요일_이름[int(round(pred_요일))]
    
    시간대_pred = [(col, train_model(df_train.drop(columns=시간대_건수_columns), df_train[col])[0].predict(df_test.drop(columns=시간대_건수_columns)).mean()) for col in 시간대_건수_columns]
    시간대_pred = sorted(시간대_pred, key=lambda x: x[1])[:3]
    휴식추천시간대 = [시간대_이름[시간대_건수_columns.index(col)] for col, _ in 시간대_pred]
    
    혼잡_예상값 = [(i, j, train_model(df_train.drop(columns=매출건수_columns), df_train[매출건수_columns[i]])[0].predict(df_test.drop(columns=매출건수_columns)).mean() +
                     train_model(df_train.drop(columns=시간대_건수_columns), df_train[시간대_건수_columns[j]])[0].predict(df_test.drop(columns=시간대_건수_columns)).mean())
                     for i in range(len(매출건수_columns)) for j in range(len(시간대_건수_columns))]
    혼잡예상시간대 = sorted(혼잡_예상값, key=lambda x: x[2], reverse=True)[:3]
    혼잡예상시간대 = [f"{요일_이름[i]} {시간대_이름[j]}" for i, j, _ in 혼잡예상시간대]
    
    return 휴무추천일, 휴식추천시간대, 혼잡예상시간대

if __name__ == "__main__":
    df = pd.read_csv("C:\\Users\\m\\Desktop\\머신러닝 사용 데이터\\피처엔지니어링한통합데이터\\피처엔지니어링일단다한통합데이터.csv")
    상권명 = input("상권명을 입력하세요: ")
    업종명 = input("업종명을 입력하세요: ")
    df_train, df_test = preprocess_data(df, 상권명, 업종명)
    휴무추천일, 휴식추천시간대, 혼잡예상시간대 = predict_sales(df_train, df_test)
    
    print(f"휴무 추천일: {휴무추천일}")
    print(f"휴식 추천 시간대 (상위 3개): {휴식추천시간대}")
    print(f"혼잡 예상 시간대 (상위 3개): {혼잡예상시간대}")

KeyError: '상권코드명'

딥시크 피드백

In [4]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

def train_model(X, y):
    X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=42)
    model = RandomForestRegressor(n_estimators=100, random_state=42)
    model.fit(X_train, y_train)
    
    y_valid_pred = model.predict(X_valid)
    mae = mean_absolute_error(y_valid, y_valid_pred)
    mse = mean_squared_error(y_valid, y_valid_pred)
    r2 = r2_score(y_valid, y_valid_pred)
    
    return model, mae, mse, r2

def preprocess_data(df, 상권명, 업종명):
    # Filter data based on '상권_코드_명' and '서비스_업종_코드_명'
    df_filtered = df[(df['상권_코드_명'] == 상권명) & (df['서비스_업종_코드_명'] == 업종명)]
    
    # Split data into training and test sets based on '기준_년분기_코드'
    df_train = df_filtered[df_filtered['기준_년분기_코드'] <= 20234]
    df_test = df_filtered[df_filtered['기준_년분기_코드'] == 20241]
    
    return df_train, df_test

def predict_sales(df_train, df_test):
    요일_columns = ['월요일_매출_금액', '화요일_매출_금액', '수요일_매출_금액', '목요일_매출_금액', '금요일_매출_금액', '토요일_매출_금액', '일요일_매출_금액']
    매출건수_columns = ['월요일_매출_건수', '화요일_매출_건수', '수요일_매출_건수', '목요일_매출_건수', '금요일_매출_건수', '토요일_매출_건수', '일요일_매출_건수']
    시간대_건수_columns = ['시간대_00~06_매출_건수', '시간대_06~11_매출_건수', '시간대_11~14_매출_건수', '시간대_14~17_매출_건수', '시간대_17~21_매출_건수', '시간대_21~24_매출_건수']
    
    요일_이름 = ['월요일', '화요일', '수요일', '목요일', '금요일', '토요일', '일요일']
    시간대_이름 = ['00~06시', '06~11시', '11~14시', '14~17시', '17~21시', '21~24시']
    
    # Calculate concentration ratios
    df_train[요일_columns] = df_train[요일_columns].div(df_train[요일_columns].sum(axis=1), axis=0)
    df_train['요일별_매출_집중도'] = df_train[요일_columns].mean(axis=1)  # 특정 값을 요약해서 할당
    df_train['시간대별_매출_집중도'] = df_train[시간대_건수_columns].div(df_train[시간대_건수_columns].sum(axis=1), axis=0)
    df_train['영업_효율성_지수'] = df_train['당월_매출_금액'] / df_train['총_유동인구_수']
    
    df_test['요일별_매출_집중도'] = df_test[요일_columns].div(df_test[요일_columns].sum(axis=1), axis=0)
    df_test['시간대별_매출_집중도'] = df_test[시간대_건수_columns].div(df_test[시간대_건수_columns].sum(axis=1), axis=0)
    df_test['영업_효율성_지수'] = df_test['당월_매출_금액'] / df_test['총_유동인구_수']
    
    # Predict recommended day off
    요일_model, _, _, _ = train_model(df_train[요일_columns], df_train[요일_columns].idxmin(axis=1).map(lambda x: 요일_columns.index(x)))
    pred_요일 = np.mean(요일_model.predict(df_test[요일_columns]))  
    휴무추천일 = 요일_이름[int(round(pred_요일))]
    
    # Predict recommended break times
    시간대_pred = [(col, train_model(df_train.drop(columns=시간대_건수_columns), df_train[col])[0].predict(df_test.drop(columns=시간대_건수_columns)).mean()) for col in 시간대_건수_columns]
    시간대_pred = sorted(시간대_pred, key=lambda x: x[1])[:3]
    휴식추천시간대 = [시간대_이름[시간대_건수_columns.index(col)] for col, _ in 시간대_pred]
    
    # Predict busy times
    혼잡_예상값 = [(i, j, train_model(df_train.drop(columns=매출건수_columns), df_train[매출건수_columns[i]])[0].predict(df_test.drop(columns=매출건수_columns)).mean() +
                     train_model(df_train.drop(columns=시간대_건수_columns), df_train[시간대_건수_columns[j]])[0].predict(df_test.drop(columns=시간대_건수_columns)).mean())
                     for i in range(len(매출건수_columns)) for j in range(len(시간대_건수_columns))]
    혼잡예상시간대 = sorted(혼잡_예상값, key=lambda x: x[2], reverse=True)[:3]
    혼잡예상시간대 = [f"{요일_이름[i]} {시간대_이름[j]}" for i, j, _ in 혼잡예상시간대]
    
    return 휴무추천일, 휴식추천시간대, 혼잡예상시간대

if __name__ == "__main__":
    df = pd.read_csv("C:\\Users\\m\\Desktop\\머신러닝 사용 데이터\\피처엔지니어링한통합데이터\\피처엔지니어링일단다한통합데이터.csv")
    
    # Debugging: Print column names
    print("Columns in DataFrame:", df.columns.tolist())
    
    # Debugging: Print first few rows
    print("First few rows of DataFrame:")
    print(df.head())
    
    상권명 = input("상권명을 입력하세요: ")
    업종명 = input("업종명을 입력하세요: ")
    
    df_train, df_test = preprocess_data(df, 상권명, 업종명)
    휴무추천일, 휴식추천시간대, 혼잡예상시간대 = predict_sales(df_train, df_test)
    
    print(f"휴무 추천일: {휴무추천일}")
    print(f"휴식 추천 시간대 (상위 3개): {휴식추천시간대}")
    print(f"혼잡 예상 시간대 (상위 3개): {혼잡예상시간대}")

Columns in DataFrame: ['기준_년분기_코드', '상권_구분_코드', '상권_구분_코드_명', '상권_코드', '상권_코드_명', '서비스_업종_코드', '서비스_업종_코드_명', '당월_매출_금액', '당월_매출_건수', '주중_매출_금액', '주말_매출_금액', '월요일_매출_금액', '화요일_매출_금액', '수요일_매출_금액', '목요일_매출_금액', '금요일_매출_금액', '토요일_매출_금액', '일요일_매출_금액', '시간대_00~06_매출_금액', '시간대_06~11_매출_금액', '시간대_11~14_매출_금액', '시간대_14~17_매출_금액', '시간대_17~21_매출_금액', '시간대_21~24_매출_금액', '남성_매출_금액', '여성_매출_금액', '연령대_10_매출_금액', '연령대_20_매출_금액', '연령대_30_매출_금액', '연령대_40_매출_금액', '연령대_50_매출_금액', '연령대_60_이상_매출_금액', '주중_매출_건수', '주말_매출_건수', '월요일_매출_건수', '화요일_매출_건수', '수요일_매출_건수', '목요일_매출_건수', '금요일_매출_건수', '토요일_매출_건수', '일요일_매출_건수', '시간대_00~06_매출_건수', '시간대_06~11_매출_건수', '시간대_11~14_매출_건수', '시간대_14~17_매출_건수', '시간대_17~21_매출_건수', '시간대_21~24_매출_건수', '남성_매출_건수', '여성_매출_건수', '연령대_10_매출_건수', '연령대_20_매출_건수', '연령대_30_매출_건수', '연령대_40_매출_건수', '연령대_50_매출_건수', '연령대_60_이상_매출_건수', '총_유동인구_수', '남성_유동인구_수', '여성_유동인구_수', '연령대_10_유동인구_수', '연령대_20_유동인구_수', '연령대_30_유동인구_수', '연령대_40_유동인구_수', '연령대_50_유동인구_수', '연령대_60_이상_유동인구_수', '시간대_00_06_유동인구_수',

ValueError: Cannot set a DataFrame with multiple columns to the single column 시간대별_매출_집중도

In [5]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

def train_model(X, y):
    X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=42)
    model = RandomForestRegressor(n_estimators=100, random_state=42)
    model.fit(X_train, y_train)
    
    y_valid_pred = model.predict(X_valid)
    mae = mean_absolute_error(y_valid, y_valid_pred)
    mse = mean_squared_error(y_valid, y_valid_pred)
    r2 = r2_score(y_valid, y_valid_pred)
    
    return model, mae, mse, r2

def preprocess_data(df, 상권명, 업종명):
    # Filter data based on '상권_코드_명' and '서비스_업종_코드_명'
    df_filtered = df[(df['상권_코드_명'] == 상권명) & (df['서비스_업종_코드_명'] == 업종명)]
    
    # Split data into training and test sets based on '기준_년분기_코드'
    df_train = df_filtered[df_filtered['기준_년분기_코드'] <= 20234]
    df_test = df_filtered[df_filtered['기준_년분기_코드'] == 20241]
    
    return df_train, df_test

def predict_sales(df_train, df_test):
    요일_columns = ['월요일_매출_금액', '화요일_매출_금액', '수요일_매출_금액', '목요일_매출_금액', '금요일_매출_금액', '토요일_매출_금액', '일요일_매출_금액']
    시간대_건수_columns = ['시간대_00~06_매출_건수', '시간대_06~11_매출_건수', '시간대_11~14_매출_건수', '시간대_14~17_매출_건수', '시간대_17~21_매출_건수', '시간대_21~24_매출_건수']
    
    요일_이름 = ['월요일', '화요일', '수요일', '목요일', '금요일', '토요일', '일요일']
    시간대_이름 = ['00~06시', '06~11시', '11~14시', '14~17시', '17~21시', '21~24시']
    
    # 매출 집중도 계산
    df_train[요일_columns] = df_train[요일_columns].div(df_train[요일_columns].sum(axis=1), axis=0)
    df_train['요일별_매출_집중도'] = df_train[요일_columns].mean(axis=1)
    df_train['시간대별_매출_집중도'] = df_train[시간대_건수_columns].div(df_train[시간대_건수_columns].sum(axis=1).replace(0, 1), axis=0).mean(axis=1)
    
    df_test['요일별_매출_집중도'] = df_test[요일_columns].div(df_test[요일_columns].sum(axis=1), axis=0).mean(axis=1)
    df_test['시간대별_매출_집중도'] = df_test[시간대_건수_columns].div(df_test[시간대_건수_columns].sum(axis=1).replace(0, 1), axis=0).mean(axis=1)

    # 영업 효율성 지수 계산
    df_train['영업_효율성_지수'] = df_train['당월_매출_금액'] / df_train['총_유동인구_수']
    df_test['영업_효율성_지수'] = df_test['당월_매출_금액'] / df_test['총_유동인구_수']
    
    # 휴무 추천일 예측
    요일_model, _, _, _ = train_model(df_train[요일_columns], df_train[요일_columns].idxmin(axis=1).map(lambda x: 요일_columns.index(x)))
    pred_요일 = np.mean(요일_model.predict(df_test[요일_columns]))  
    휴무추천일 = 요일_이름[int(round(pred_요일))]
    
    # 휴식 추천 시간대 예측
    시간대_pred = [(col, train_model(df_train.drop(columns=시간대_건수_columns), df_train[col])[0].predict(df_test.drop(columns=시간대_건수_columns)).mean()) for col in 시간대_건수_columns]
    시간대_pred = sorted(시간대_pred, key=lambda x: x[1])[:3]
    휴식추천시간대 = [시간대_이름[시간대_건수_columns.index(col)] for col, _ in 시간대_pred]
    
    # 혼잡 예상 시간대 예측
    혼잡_예상값 = [(i, j, train_model(df_train.drop(columns=시간대_건수_columns), df_train[시간대_건수_columns[j]])[0].predict(df_test.drop(columns=시간대_건수_columns)).mean())
                   for i in range(len(요일_columns)) for j in range(len(시간대_건수_columns))]
    혼잡예상시간대 = sorted(혼잡_예상값, key=lambda x: x[2], reverse=True)[:3]
    혼잡예상시간대 = [f"{요일_이름[i]} {시간대_이름[j]}" for i, j, _ in 혼잡예상시간대]
    
    return 휴무추천일, 휴식추천시간대, 혼잡예상시간대

if __name__ == "__main__":
    df = pd.read_csv("C:\\Users\\m\\Desktop\\머신러닝 사용 데이터\\피처엔지니어링한통합데이터\\피처엔지니어링일단다한통합데이터.csv")
    
    # Debugging: Print column names
    print("Columns in DataFrame:", df.columns.tolist())
    
    # Debugging: Print first few rows
    print("First few rows of DataFrame:")
    print(df.head())
    
    상권명 = input("상권명을 입력하세요: ")
    업종명 = input("업종명을 입력하세요: ")
    
    df_train, df_test = preprocess_data(df, 상권명, 업종명)
    휴무추천일, 휴식추천시간대, 혼잡예상시간대 = predict_sales(df_train, df_test)
    
    print(f"휴무 추천일: {휴무추천일}")
    print(f"휴식 추천 시간대 (상위 3개): {휴식추천시간대}")
    print(f"혼잡 예상 시간대 (상위 3개): {혼잡예상시간대}")

Columns in DataFrame: ['기준_년분기_코드', '상권_구분_코드', '상권_구분_코드_명', '상권_코드', '상권_코드_명', '서비스_업종_코드', '서비스_업종_코드_명', '당월_매출_금액', '당월_매출_건수', '주중_매출_금액', '주말_매출_금액', '월요일_매출_금액', '화요일_매출_금액', '수요일_매출_금액', '목요일_매출_금액', '금요일_매출_금액', '토요일_매출_금액', '일요일_매출_금액', '시간대_00~06_매출_금액', '시간대_06~11_매출_금액', '시간대_11~14_매출_금액', '시간대_14~17_매출_금액', '시간대_17~21_매출_금액', '시간대_21~24_매출_금액', '남성_매출_금액', '여성_매출_금액', '연령대_10_매출_금액', '연령대_20_매출_금액', '연령대_30_매출_금액', '연령대_40_매출_금액', '연령대_50_매출_금액', '연령대_60_이상_매출_금액', '주중_매출_건수', '주말_매출_건수', '월요일_매출_건수', '화요일_매출_건수', '수요일_매출_건수', '목요일_매출_건수', '금요일_매출_건수', '토요일_매출_건수', '일요일_매출_건수', '시간대_00~06_매출_건수', '시간대_06~11_매출_건수', '시간대_11~14_매출_건수', '시간대_14~17_매출_건수', '시간대_17~21_매출_건수', '시간대_21~24_매출_건수', '남성_매출_건수', '여성_매출_건수', '연령대_10_매출_건수', '연령대_20_매출_건수', '연령대_30_매출_건수', '연령대_40_매출_건수', '연령대_50_매출_건수', '연령대_60_이상_매출_건수', '총_유동인구_수', '남성_유동인구_수', '여성_유동인구_수', '연령대_10_유동인구_수', '연령대_20_유동인구_수', '연령대_30_유동인구_수', '연령대_40_유동인구_수', '연령대_50_유동인구_수', '연령대_60_이상_유동인구_수', '시간대_00_06_유동인구_수',

ValueError: could not convert string to float: 'D'

In [7]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.preprocessing import LabelEncoder

# 모든 문자열 또는 범주형 열에 대해 Label Encoding 수행
le = LabelEncoder()
for col in df_train.columns:
    if df_train[col].dtype == 'object':
        df_train[col] = le.fit_transform(df_train[col])
        df_test[col] = le.transform(df_test[col])

def train_model(X, y):
    X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=42)
    model = RandomForestRegressor(n_estimators=100, random_state=42)
    model.fit(X_train, y_train)
    
    y_valid_pred = model.predict(X_valid)
    mae = mean_absolute_error(y_valid, y_valid_pred)
    mse = mean_squared_error(y_valid, y_valid_pred)
    r2 = r2_score(y_valid, y_valid_pred)
    
    return model, mae, mse, r2

def preprocess_data(df, 상권명, 업종명):
    # Filter data based on '상권_코드_명' and '서비스_업종_코드_명'
    df_filtered = df[(df['상권_코드_명'] == 상권명) & (df['서비스_업종_코드_명'] == 업종명)]
    
    # Split data into training and test sets based on '기준_년분기_코드'
    df_train = df_filtered[df_filtered['기준_년분기_코드'] <= 20234]
    df_test = df_filtered[df_filtered['기준_년분기_코드'] == 20241]
    
    return df_train, df_test

def predict_sales(df_train, df_test):
    요일_columns = ['월요일_매출_금액', '화요일_매출_금액', '수요일_매출_금액', '목요일_매출_금액', '금요일_매출_금액', '토요일_매출_금액', '일요일_매출_금액']
    시간대_건수_columns = ['시간대_00~06_매출_건수', '시간대_06~11_매출_건수', '시간대_11~14_매출_건수', '시간대_14~17_매출_건수', '시간대_17~21_매출_건수', '시간대_21~24_매출_건수']
    
    요일_이름 = ['월요일', '화요일', '수요일', '목요일', '금요일', '토요일', '일요일']
    시간대_이름 = ['00~06시', '06~11시', '11~14시', '14~17시', '17~21시', '21~24시']
    
    # 매출 집중도 계산
    df_train[요일_columns] = df_train[요일_columns].div(df_train[요일_columns].sum(axis=1), axis=0)
    df_train['요일별_매출_집중도'] = df_train[요일_columns].mean(axis=1)
    df_train['시간대별_매출_집중도'] = df_train[시간대_건수_columns].div(df_train[시간대_건수_columns].sum(axis=1).replace(0, 1), axis=0).mean(axis=1)
    
    df_test['요일별_매출_집중도'] = df_test[요일_columns].div(df_test[요일_columns].sum(axis=1), axis=0).mean(axis=1)
    df_test['시간대별_매출_집중도'] = df_test[시간대_건수_columns].div(df_test[시간대_건수_columns].sum(axis=1).replace(0, 1), axis=0).mean(axis=1)

    # 영업 효율성 지수 계산
    df_train['영업_효율성_지수'] = df_train['당월_매출_금액'] / df_train['총_유동인구_수']
    df_test['영업_효율성_지수'] = df_test['당월_매출_금액'] / df_test['총_유동인구_수']
    
    # 휴무 추천일 예측
    요일_model, _, _, _ = train_model(df_train[요일_columns], df_train[요일_columns].idxmin(axis=1).map(lambda x: 요일_columns.index(x)))
    pred_요일 = np.mean(요일_model.predict(df_test[요일_columns]))  
    휴무추천일 = 요일_이름[int(round(pred_요일))]
    
    # 휴식 추천 시간대 예측
    시간대_pred = [(col, train_model(df_train.drop(columns=시간대_건수_columns), df_train[col])[0].predict(df_test.drop(columns=시간대_건수_columns)).mean()) for col in 시간대_건수_columns]
    시간대_pred = sorted(시간대_pred, key=lambda x: x[1])[:3]
    휴식추천시간대 = [시간대_이름[시간대_건수_columns.index(col)] for col, _ in 시간대_pred]
    
    # 혼잡 예상 시간대 예측
    혼잡_예상값 = [(i, j, train_model(df_train.drop(columns=시간대_건수_columns), df_train[시간대_건수_columns[j]])[0].predict(df_test.drop(columns=시간대_건수_columns)).mean())
                   for i in range(len(요일_columns)) for j in range(len(시간대_건수_columns))]
    혼잡예상시간대 = sorted(혼잡_예상값, key=lambda x: x[2], reverse=True)[:3]
    혼잡예상시간대 = [f"{요일_이름[i]} {시간대_이름[j]}" for i, j, _ in 혼잡예상시간대]
    
    return 휴무추천일, 휴식추천시간대, 혼잡예상시간대

if __name__ == "__main__":
    df = pd.read_csv("C:\\Users\\m\\Desktop\\머신러닝 사용 데이터\\피처엔지니어링한통합데이터\\피처엔지니어링일단다한통합데이터.csv")
    
    # Debugging: Print column names
    print("Columns in DataFrame:", df.columns.tolist())
    
    # Debugging: Print first few rows
    print("First few rows of DataFrame:")
    print(df.head())
    
    상권명 = input("상권명을 입력하세요: ")
    업종명 = input("업종명을 입력하세요: ")
    
    df_train, df_test = preprocess_data(df, 상권명, 업종명)
    휴무추천일, 휴식추천시간대, 혼잡예상시간대 = predict_sales(df_train, df_test)
    
    print(f"휴무 추천일: {휴무추천일}")
    print(f"휴식 추천 시간대 (상위 3개): {휴식추천시간대}")
    print(f"혼잡 예상 시간대 (상위 3개): {혼잡예상시간대}")

Columns in DataFrame: ['기준_년분기_코드', '상권_구분_코드', '상권_구분_코드_명', '상권_코드', '상권_코드_명', '서비스_업종_코드', '서비스_업종_코드_명', '당월_매출_금액', '당월_매출_건수', '주중_매출_금액', '주말_매출_금액', '월요일_매출_금액', '화요일_매출_금액', '수요일_매출_금액', '목요일_매출_금액', '금요일_매출_금액', '토요일_매출_금액', '일요일_매출_금액', '시간대_00~06_매출_금액', '시간대_06~11_매출_금액', '시간대_11~14_매출_금액', '시간대_14~17_매출_금액', '시간대_17~21_매출_금액', '시간대_21~24_매출_금액', '남성_매출_금액', '여성_매출_금액', '연령대_10_매출_금액', '연령대_20_매출_금액', '연령대_30_매출_금액', '연령대_40_매출_금액', '연령대_50_매출_금액', '연령대_60_이상_매출_금액', '주중_매출_건수', '주말_매출_건수', '월요일_매출_건수', '화요일_매출_건수', '수요일_매출_건수', '목요일_매출_건수', '금요일_매출_건수', '토요일_매출_건수', '일요일_매출_건수', '시간대_00~06_매출_건수', '시간대_06~11_매출_건수', '시간대_11~14_매출_건수', '시간대_14~17_매출_건수', '시간대_17~21_매출_건수', '시간대_21~24_매출_건수', '남성_매출_건수', '여성_매출_건수', '연령대_10_매출_건수', '연령대_20_매출_건수', '연령대_30_매출_건수', '연령대_40_매출_건수', '연령대_50_매출_건수', '연령대_60_이상_매출_건수', '총_유동인구_수', '남성_유동인구_수', '여성_유동인구_수', '연령대_10_유동인구_수', '연령대_20_유동인구_수', '연령대_30_유동인구_수', '연령대_40_유동인구_수', '연령대_50_유동인구_수', '연령대_60_이상_유동인구_수', '시간대_00_06_유동인구_수',

ValueError: could not convert string to float: 'D'

In [9]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error

def load_data():
    # 예제용 데이터 로드 (여기에 사용자 데이터 경로 입력)
    data = {
        '요일': ['월', '화', '수', '목', '금', '토', '일'],
        '시간대_09_12': [100, 120, 110, 130, 150, 200, 180],
        '시간대_12_15': [140, 160, 170, 150, 200, 250, 220],
        '시간대_15_18': [90, 100, 80, 120, 140, 180, 170],
        '업종명': ['카페', '카페', '식당', '카페', '식당', '카페', '식당'],
        '상권명': ['A', 'A', 'A', 'A', 'A', 'A', 'A']
    }
    return pd.DataFrame(data)

def preprocess_data(df, 상권명, 업종명):
    # 상권명 및 업종명 필터링
    df = df[(df['상권명'] == 상권명) & (df['업종명'] == 업종명)]
    
    # 요일 및 시간대 컬럼
    요일_columns = ['요일']
    시간대_건수_columns = ['시간대_09_12', '시간대_12_15', '시간대_15_18']

    # One-Hot-Encoding으로 범주형 데이터를 수치형으로 변환
    df = pd.get_dummies(df, columns=요일_columns, drop_first=True)

    # 학습/테스트 데이터 분리
    df_train = df.iloc[:-2]  # 마지막 2개는 테스트용
    df_test = df.iloc[-2:]

    return df_train, df_test

def train_model(X, y):
    # 학습 및 검증 데이터 분리
    X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=42)
    model = RandomForestRegressor(n_estimators=100, random_state=42)
    
    model.fit(X_train, y_train)
    
    y_valid_pred = model.predict(X_valid)
    mae = mean_absolute_error(y_valid, y_valid_pred)
    return model, mae

def predict_sales(df_train, df_test):
    시간대_건수_columns = ['시간대_09_12', '시간대_12_15', '시간대_15_18']
    시간대_이름 = ['09-12시', '12-15시', '15-18시']

    # 컬럼 정렬로 train/test 데이터 일치 보장
    df_train, df_test = df_train.align(df_test, axis=1, fill_value=0)

    # 요일 예측 모델
    요일_target = [col for col in df_train.columns if col.startswith('요일_')]
    요일_model, _ = train_model(df_train.drop(columns=시간대_건수_columns), df_train[요일_target].sum(axis=1))

    pred_요일 = np.mean(요일_model.predict(df_test.drop(columns=시간대_건수_columns)))
    요일_이름 = ['월', '화', '수', '목', '금', '토', '일']
    휴무추천일 = 요일_이름[int(round(pred_요일)) % len(요일_이름)]

    # 휴식 추천 시간대 예측
    시간대_pred = [(col, train_model(df_train.drop(columns=시간대_건수_columns), df_train[col])[0]
                    .predict(df_test.drop(columns=시간대_건수_columns)).mean()) for col in 시간대_건수_columns]
    시간대_pred = sorted(시간대_pred, key=lambda x: x[1])[:3]
    휴식추천시간대 = [시간대_이름[시간대_건수_columns.index(col)] for col, _ in 시간대_pred]

    # 혼잡 예상 시간대 (상위 1개)
    혼잡예상시간대 = max(시간대_pred, key=lambda x: x[1])[0]

    return 휴무추천일, 휴식추천시간대, 혼잡예상시간대

# 실행 코드
df = load_data()
상권명 = input("상권명을 입력하세요: ")
업종명 = input("업종명을 입력하세요: ")

df_train, df_test = preprocess_data(df, 상권명, 업종명)
휴무추천일, 휴식추천시간대, 혼잡예상시간대 = predict_sales(df_train, df_test)

print(f"휴무 추천일: {휴무추천일}")
print(f"휴식 추천 시간대 (상위 3개): {휴식추천시간대}")
print(f"혼잡 예상 시간대: {혼잡예상시간대}")

ValueError: With n_samples=0, test_size=0.2 and train_size=None, the resulting train set will be empty. Adjust any of the aforementioned parameters.

GPT 조언

In [10]:
import pandas as pd

def preprocess_data(df, 상권명, 업종명):
    df_filtered = df[(df['상권_코드_명'] == 상권명) & (df['서비스_업종_코드_명'] == 업종명)]
    df_train = df_filtered[df_filtered['기준_년분기_코드'] <= 20234]
    df_test = df_filtered[df_filtered['기준_년분기_코드'] == 20241]
    return df_train, df_test

def predict_sales(df_train, df_test):
    요일별_매출 = {
        '월요일': '월요일_매출_금액', '화요일': '화요일_매출_금액', '수요일': '수요일_매출_금액',
        '목요일': '목요일_매출_금액', '금요일': '금요일_매출_금액',
        '토요일': '토요일_매출_금액', '일요일': '일요일_매출_금액'
    }
    시간대별_매출 = {
        '00~06': '시간대_00~06_매출_금액', '06~11': '시간대_06~11_매출_금액',
        '11~14': '시간대_11~14_매출_금액', '14~17': '시간대_14~17_매출_금액',
        '17~21': '시간대_17~21_매출_금액', '21~24': '시간대_21~24_매출_금액'
    }
    
    요일_평균 = {k: df_train[v].mean() for k, v in 요일별_매출.items()}
    시간대_평균 = {k: df_train[v].mean() for k, v in 시간대별_매출.items()}
    
    휴무추천일 = min(요일_평균, key=요일_평균.get)
    휴식추천시간대 = min(시간대_평균, key=시간대_평균.get)
    
    혼잡_시간대별_매출 = sorted(시간대_평균.items(), key=lambda x: x[1], reverse=True)[:3]
    혼잡_요일별_매출 = sorted(요일_평균.items(), key=lambda x: x[1], reverse=True)[:3]
    혼잡예상시간대 = [f"{d[0]}_{t[0]}" for d, t in zip(혼잡_요일별_매출, 혼잡_시간대별_매출)]
    
    return 휴무추천일, 휴식추천시간대, 혼잡예상시간대

df = pd.read_csv("C:\\Users\\m\\Desktop\\머신러닝 사용 데이터\\피처엔지니어링한통합데이터\\피처엔지니어링일단다한통합데이터.csv")

상권명 = input("상권명을 입력하세요: ")
업종명 = input("업종명을 입력하세요: ")

df_train, df_test = preprocess_data(df, 상권명, 업종명)
휴무추천일, 휴식추천시간대, 혼잡예상시간대 = predict_sales(df_train, df_test)

print(f"휴무 추천일: {휴무추천일}")
print(f"휴식 추천 시간대: {휴식추천시간대}")
print(f"혼잡 예상 시간대: {', '.join(혼잡예상시간대)}")

휴무 추천일: 일요일
휴식 추천 시간대: 00~06
혼잡 예상 시간대: 화요일_17~21, 금요일_14~17, 월요일_21~24


In [11]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error

def preprocess_data(df, 상권명, 업종명):
    df_filtered = df[(df['상권_코드_명'] == 상권명) & (df['서비스_업종_코드_명'] == 업종명)]
    df_train = df_filtered[df_filtered['기준_년분기_코드'] <= 20234]
    df_test = df_filtered[df_filtered['기준_년분기_코드'] == 20241]
    return df_train, df_test

def predict_sales(df_train, df_test):
    요일_컬럼 = ['월요일_매출_금액', '화요일_매출_금액', '수요일_매출_금액', '목요일_매출_금액', '금요일_매출_금액', '토요일_매출_금액', '일요일_매출_금액']
    시간대_컬럼 = ['시간대_00~06_매출_금액', '시간대_06~11_매출_금액', '시간대_11~14_매출_금액', '시간대_14~17_매출_금액', '시간대_17~21_매출_금액', '시간대_21~24_매출_금액']
    
    X_train = df_train[요일_컬럼 + 시간대_컬럼]
    y_train = df_train['당월_매출_금액']
    X_test = df_test[요일_컬럼 + 시간대_컬럼]
    y_test = df_test['당월_매출_금액']
    
    model = RandomForestRegressor(n_estimators=100, random_state=42)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    
    mae = mean_absolute_error(y_test, y_pred)
    mse = mean_squared_error(y_test, y_pred)
    
    요일별_평균매출 = X_test[요일_컬럼].mean()
    시간대별_평균매출 = X_test[시간대_컬럼].mean()
    
    휴무추천일 = 요일별_평균매출.idxmin().replace('_매출_금액', '')
    휴식추천시간대 = 시간대별_평균매출.idxmin().replace('시간대_', '').replace('_매출_금액', '시')
    
    요일_시간대_조합 = [(요일.replace('_매출_금액', ''), 시간.replace('시간대_', '').replace('_매출_금액', '시')) for 요일 in 요일_컬럼 for 시간 in 시간대_컬럼]
    df_comb = pd.DataFrame(요일_시간대_조합, columns=['요일', '시간대'])
    df_comb['매출'] = [X_test[요일].values[0] + X_test[시간].values[0] for 요일, 시간 in 요일_시간대_조합]
    혼잡예상시간대 = df_comb.nlargest(3, '매출').apply(lambda row: f"{row['요일']} {row['시간대']}", axis=1).tolist()
    
    return 휴무추천일, 휴식추천시간대, 혼잡예상시간대, mae, mse

# 데이터 불러오기
df = pd.read_csv("C:\\Users\\m\\Desktop\\머신러닝 사용 데이터\\피처엔지니어링한통합데이터\\피처엔지니어링일단다한통합데이터.csv")

상권명 = input("상권명을 입력하세요: ")
업종명 = input("업종명을 입력하세요: ")

df_train, df_test = preprocess_data(df, 상권명, 업종명)
휴무추천일, 휴식추천시간대, 혼잡예상시간대, mae, mse = predict_sales(df_train, df_test)

print(f"휴무 추천일: {휴무추천일}")
print(f"휴식 추천 시간대: {휴식추천시간대}")
print(f"혼잡 예상 시간대: {', '.join(혼잡예상시간대)}")
print(f"MAE: {mae:.2f}")
print(f"MSE: {mse:.2f}")

KeyError: '월요일'

In [12]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error

# 데이터 불러오기
df = pd.read_csv("C:\\Users\\m\\Desktop\\머신러닝 사용 데이터\\피처엔지니어링한통합데이터\\피처엔지니어링일단다한통합데이터.csv")

def preprocess_data(df, 상권명, 업종명):
    df_filtered = df[(df['상권_코드_명'] == 상권명) & (df['서비스_업종_코드_명'] == 업종명)]
    feature_cols = ['월요일_매출_금액', '화요일_매출_금액', '수요일_매출_금액', '목요일_매출_금액', '금요일_매출_금액',
                    '토요일_매출_금액', '일요일_매출_금액', '시간대_00~06_매출_금액', '시간대_06~11_매출_금액', '시간대_11~14_매출_금액',
                    '시간대_14~17_매출_금액', '시간대_17~21_매출_금액', '시간대_21~24_매출_금액']
    df_filtered = df_filtered[feature_cols]
    df_train, df_test = train_test_split(df_filtered, test_size=0.2, random_state=42)
    return df_train, df_test

def predict_sales(df_train, df_test):
    X_train, y_train = df_train.drop(columns=['월요일_매출_금액']), df_train['월요일_매출_금액']
    X_test, y_test = df_test.drop(columns=['월요일_매출_금액']), df_test['월요일_매출_금액']
    
    model = RandomForestRegressor(n_estimators=100, random_state=42)
    model.fit(X_train, y_train)
    
    y_pred = model.predict(X_test)
    mae = mean_absolute_error(y_test, y_pred)
    mse = mean_squared_error(y_test, y_pred)
    
    요일_컬럼 = ['월요일_매출_금액', '화요일_매출_금액', '수요일_매출_금액', '목요일_매출_금액', '금요일_매출_금액', '토요일_매출_금액', '일요일_매출_금액']
    시간대_컬럼 = ['시간대_00~06_매출_금액', '시간대_06~11_매출_금액', '시간대_11~14_매출_금액', '시간대_14~17_매출_금액', '시간대_17~21_매출_금액', '시간대_21~24_매출_금액']
    
    휴무추천일 = min(요일_컬럼, key=lambda x: X_test[x].mean()).replace('_매출_금액', '')
    휴식추천시간대 = min(시간대_컬럼, key=lambda x: X_test[x].mean()).replace('시간대_', '').replace('_매출_금액', '시')
    
    요일_시간대_조합 = [(요일, 시간) for 요일 in 요일_컬럼 for 시간 in 시간대_컬럼]
    df_comb = pd.DataFrame(요일_시간대_조합, columns=['요일', '시간대'])
    df_comb['매출'] = [X_test[요일].values[0] + X_test[시간].values[0] for 요일, 시간 in 요일_시간대_조합]
    혼잡예상시간대 = df_comb.nlargest(3, '매출').apply(lambda row: f"{row['요일'].replace('_매출_금액', '')} {row['시간대'].replace('시간대_', '').replace('_매출_금액', '시')}", axis=1).tolist()
    
    return 휴무추천일, 휴식추천시간대, 혼잡예상시간대, mae, mse

# 사용자 입력
상권명 = input("상권명을 입력하세요: ")
업종명 = input("업종명을 입력하세요: ")

df_train, df_test = preprocess_data(df, 상권명, 업종명)
휴무추천일, 휴식추천시간대, 혼잡예상시간대, mae, mse = predict_sales(df_train, df_test)

print(f"휴무 추천일: {휴무추천일}")
print(f"휴식 추천 시간대: {휴식추천시간대}")
print(f"혼잡 예상 시간대: {', '.join(혼잡예상시간대)}")
print(f"MAE: {mae:.2f}, MSE: {mse:.2f}")

KeyError: '월요일_매출_금액'

In [19]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error

# 데이터 불러오기
df = pd.read_csv("C:\\Users\\m\\Desktop\\머신러닝 사용 데이터\\피처엔지니어링한통합데이터\\피처엔지니어링일단다한통합데이터.csv")

def preprocess_data(df, 상권명, 업종명):
    df_filtered = df[(df['상권_코드_명'] == 상권명) & (df['서비스_업종_코드_명'] == 업종명)]
    feature_cols = [
        '월요일_매출_금액', '화요일_매출_금액', '수요일_매출_금액', '목요일_매출_금액', '금요일_매출_금액',
        '토요일_매출_금액', '일요일_매출_금액', '시간대_00~06_매출_금액', '시간대_06~11_매출_금액', '시간대_11~14_매출_금액',
        '시간대_14~17_매출_금액', '시간대_17~21_매출_금액', '시간대_21~24_매출_금액'
    ]
    df_filtered = df_filtered[feature_cols].dropna()
    df_train, df_test = train_test_split(df_filtered, test_size=0.2, random_state=42)
    return df_train, df_test

def predict_sales(df_train, df_test):
    X_train, y_train = df_train.drop(columns=['월요일_매출_금액']), df_train['월요일_매출_금액']
    X_test, y_test = df_test.drop(columns=['월요일_매출_금액']), df_test['월요일_매출_금액']
    
    model = RandomForestRegressor(n_estimators=100, random_state=42)
    model.fit(X_train, y_train)
    
    y_pred = model.predict(X_test)
    mae = mean_absolute_error(y_test, y_pred)
    mse = mean_squared_error(y_test, y_pred)
    
    요일_컬럼 = [col for col in df_train.columns if '매출_금액' in col and any(day in col for day in ['월요일', '화요일', '수요일', '목요일', '금요일', '토요일', '일요일'])]
    시간대_컬럼 = [col for col in df_train.columns if '매출_금액' in col and '시간대' in col]
    
    휴무추천일 = min(요일_컬럼, key=lambda x: df_test[x].mean()).replace('_매출_금액', '')
    휴식추천시간대 = min(시간대_컬럼, key=lambda x: df_test[x].mean()).replace('시간대_', '').replace('_매출_금액', '시')
    
    요일_시간대_조합 = [(요일, 시간) for 요일 in 요일_컬럼 for 시간 in 시간대_컬럼]
    df_comb = pd.DataFrame(요일_시간대_조합, columns=['요일', '시간대'])
    df_comb['매출'] = [df_test.iloc[0][요일] + df_test.iloc[0][시간] for 요일, 시간 in 요일_시간대_조합]
    혼잡예상시간대 = df_comb.nlargest(3, '매출').apply(lambda row: f"{row['요일'].replace('_매출_금액', '')} {row['시간대'].replace('시간대_', '').replace('_매출_금액', '시')}", axis=1).tolist()
    
    return 휴무추천일, 휴식추천시간대, 혼잡예상시간대, mae, mse

# 사용자 입력
상권명 = input("상권명을 입력하세요: ")
업종명 = input("업종명을 입력하세요: ")

df_train, df_test = preprocess_data(df, 상권명, 업종명)
휴무추천일, 휴식추천시간대, 혼잡예상시간대, mae, mse = predict_sales(df_train, df_test)

print(f"휴무 추천일: {휴무추천일}")
print(f"휴식 추천 시간대: {휴식추천시간대}")
print(f"혼잡 예상 시간대: {', '.join(혼잡예상시간대)}")
print(f"MAE: {mae:.2f}, MSE: {mse:.2f}")

휴무 추천일: 일요일
휴식 추천 시간대: 00~06시
혼잡 예상 시간대: 수요일 17~21시, 월요일 17~21시, 목요일 17~21시
MAE: 2637102.19, MSE: 10830918168576.37


In [21]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error

# 데이터 불러오기
df = pd.read_csv("C:\\Users\\m\\Desktop\\머신러닝 사용 데이터\\피처엔지니어링한통합데이터\\피처엔지니어링일단다한통합데이터.csv")

def preprocess_data(df, 상권명, 업종명):
    df_filtered = df[(df['상권_코드_명'] == 상권명) & (df['서비스_업종_코드_명'] == 업종명)]
    feature_cols = [
        '월요일_매출_금액', '화요일_매출_금액', '수요일_매출_금액', '목요일_매출_금액', '금요일_매출_금액',
        '토요일_매출_금액', '일요일_매출_금액', '시간대_00~06_매출_금액', '시간대_06~11_매출_금액', '시간대_11~14_매출_금액',
        '시간대_14~17_매출_금액', '시간대_17~21_매출_금액', '시간대_21~24_매출_금액'
    ]
    df_filtered = df_filtered[feature_cols].dropna()
    df_train, df_test = train_test_split(df_filtered, test_size=0.2, random_state=42)
    return df_train, df_test

def predict_sales(df_train, df_test):
    X_train, y_train = df_train.drop(columns=['월요일_매출_금액']), df_train['월요일_매출_금액']
    X_test, y_test = df_test.drop(columns=['월요일_매출_금액']), df_test['월요일_매출_금액']
    
    model = RandomForestRegressor(n_estimators=100, random_state=42)
    model.fit(X_train, y_train)
    
    y_pred = model.predict(X_test)
    mae = mean_absolute_error(y_test, y_pred)
    mse = mean_squared_error(y_test, y_pred)
    
    요일_컬럼 = [col for col in df_train.columns if '매출_금액' in col and any(day in col for day in ['월요일', '화요일', '수요일', '목요일', '금요일', '토요일', '일요일'])]
    시간대_컬럼 = [col for col in df_train.columns if '매출_금액' in col and '시간대' in col]
    
    휴무추천일 = min(요일_컬럼, key=lambda x: df_test[x].mean()).replace('_매출_금액', '')
    휴식추천시간대 = min(시간대_컬럼, key=lambda x: df_test[x].mean()).replace('시간대_', '').replace('_매출_금액', '시')
    
    요일_시간대_조합 = [(요일, 시간) for 요일 in 요일_컬럼 for 시간 in 시간대_컬럼]
    df_comb = pd.DataFrame(요일_시간대_조합, columns=['요일', '시간대'])
    df_comb['매출'] = df_comb.apply(lambda row: df_test[row['요일']].mean() + df_test[row['시간대']].mean(), axis=1)
    혼잡예상시간대 = df_comb.nlargest(3, '매출').apply(lambda row: f"{row['요일'].replace('_매출_금액', '')} {row['시간대'].replace('시간대_', '').replace('_매출_금액', '시')}", axis=1).tolist()
    
    return 휴무추천일, 휴식추천시간대, 혼잡예상시간대, mae, mse

# 사용자 입력
상권명 = input("상권명을 입력하세요: ")
업종명 = input("업종명을 입력하세요: ")

df_train, df_test = preprocess_data(df, 상권명, 업종명)
휴무추천일, 휴식추천시간대, 혼잡예상시간대, mae, mse = predict_sales(df_train, df_test)

print(f"휴무 추천일: {휴무추천일}")
print(f"휴식 추천 시간대: {휴식추천시간대}")
print(f"혼잡 예상 시간대: {', '.join(혼잡예상시간대)}")
print(f"MAE: {mae:.2f}, MSE: {mse:.2f}")

휴무 추천일: 일요일
휴식 추천 시간대: 00~06시
혼잡 예상 시간대: 화요일 17~21시, 수요일 17~21시, 월요일 17~21시
MAE: 2637102.19, MSE: 10830918168576.37


In [25]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error

# 데이터 불러오기
df = pd.read_csv("C:\\Users\\m\\Desktop\\머신러닝 사용 데이터\\피처엔지니어링한통합데이터\\피처엔지니어링일단다한통합데이터.csv")

def preprocess_data(df, 상권명, 업종명):
    df_filtered = df[(df['상권_코드_명'] == 상권명) & (df['서비스_업종_코드_명'] == 업종명)]
    feature_cols = [
        '월요일_매출_건수', '화요일_매출_건수', '수요일_매출_건수', '목요일_매출_건수', '금요일_매출_건수',
        '토요일_매출_건수', '일요일_매출_건수', '시간대_00~06_매출_건수', '시간대_06~11_매출_건수', '시간대_11~14_매출_건수',
        '시간대_14~17_매출_건수', '시간대_17~21_매출_건수', '시간대_21~24_매출_건수'
    ]
    df_filtered = df_filtered[feature_cols].dropna()
    df_train, df_test = train_test_split(df_filtered, test_size=0.2, random_state=42)
    return df_train, df_test

def predict_sales(df_train, df_test):
    X_train, y_train = df_train.drop(columns=['월요일_매출_건수']), df_train['월요일_매출_건수']
    X_test, y_test = df_test.drop(columns=['월요일_매출_건수']), df_test['월요일_매출_건수']
    
    model = RandomForestRegressor(n_estimators=100, random_state=42)
    model.fit(X_train, y_train)
    
    y_pred = model.predict(X_test)
    mae = mean_absolute_error(y_test, y_pred)
    mse = mean_squared_error(y_test, y_pred)
    
    요일_컬럼 = [col for col in df_train.columns if '매출_건수' in col and any(day in col for day in ['월요일', '화요일', '수요일', '목요일', '금요일', '토요일', '일요일'])]
    시간대_컬럼 = [col for col in df_train.columns if '매출_건수' in col and '시간대' in col]
    
    # 휴무 추천일 (매출 건수가 가장 적은 요일)
    휴무추천일 = min(요일_컬럼, key=lambda x: df_test[x].mean()).replace('_매출_건수', '')

    # 휴식 추천 시간대 (매출 건수가 적은 3개의 시간대)
    시간대_평균_건수 = {시간대: df_test[시간대].mean() for 시간대 in 시간대_컬럼}
    휴식추천시간대 = sorted(시간대_평균_건수, key=시간대_평균_건수.get)[:3]
    휴식추천시간대 = [시간.replace('시간대_', '').replace('_매출_건수', '시') for 시간 in 휴식추천시간대]

    # 요일과 시간대 조합별 매출 건수 계산
    혼잡도_조합 = []
    for 요일 in 요일_컬럼:
        for 시간대 in 시간대_컬럼:
            평균_건수 = df_test[요일].mean() + df_test[시간대].mean()
            혼잡도_조합.append((요일, 시간대, 평균_건수))

    # 상위 3개 (매출 건수가 많은 조합)
    혼잡도_조합.sort(key=lambda x: x[2], reverse=True)
    혼잡예상시간대 = [f"{요일.replace('_매출_건수', '')} {시간대.replace('시간대_', '').replace('_매출_건수', '시')}" for 요일, 시간대, _ in 혼잡도_조합[:3]]

    return 휴무추천일, 휴식추천시간대, 혼잡예상시간대, mae, mse

# 사용자 입력
상권명 = input("상권명을 입력하세요: ")
업종명 = input("업종명을 입력하세요: ")

df_train, df_test = preprocess_data(df, 상권명, 업종명)
휴무추천일, 휴식추천시간대, 혼잡예상시간대, mae, mse = predict_sales(df_train, df_test)

print(f"휴무 추천일: {휴무추천일}")
print(f"휴식 추천 시간대: {', '.join(휴식추천시간대)}")
print(f"혼잡 예상 시간대: {', '.join(혼잡예상시간대)}")
print(f"MAE: {mae:.2f}, MSE: {mse:.2f}")

휴무 추천일: 일요일
휴식 추천 시간대: 00~06시, 06~11시, 21~24시
혼잡 예상 시간대: 화요일 17~21시, 수요일 17~21시, 목요일 17~21시
MAE: 8.49, MSE: 84.72


In [31]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error

# 데이터 전처리 함수
def preprocess_data(df, 상권명, 업종명):
    df_filtered = df[(df['상권_코드_명'] == 상권명) & (df['서비스_업종_코드_명'] == 업종명)]
    df_train = df_filtered[df_filtered['기준_년분기_코드'] <= 20234]  # 2023년까지 학습
    df_test = df_filtered[df_filtered['기준_년분기_코드'] == 20241]  # 2024년 1분기로 검증
    return df_train, df_test

# 모델 학습 함수
def train_model(X, y):
    X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=42)
    model = RandomForestRegressor(n_estimators=100, random_state=42)
    model.fit(X_train, y_train)

    y_valid_pred = model.predict(X_valid)
    y_valid_pred = np.nan_to_num(y_valid_pred, nan=np.mean(y_valid_pred))  # NaN이면 평균값으로 대체

    mae = mean_absolute_error(y_valid, y_valid_pred)
    mse = mean_squared_error(y_valid, y_valid_pred)

    return model, mae, mse

# 매출 예측 함수
def predict_sales(df_train, df_test):
    요일_columns = ['월요일_매출_금액', '화요일_매출_금액', '수요일_매출_금액', '목요일_매출_금액', '금요일_매출_금액', '토요일_매출_금액', '일요일_매출_금액']
    시간대_columns = ['시간대_00~06_매출_금액', '시간대_06~11_매출_금액', '시간대_11~14_매출_금액', '시간대_14~17_매출_금액', '시간대_17~21_매출_금액', '시간대_21~24_매출_금액']
    
    요일_이름 = ['월요일', '화요일', '수요일', '목요일', '금요일', '토요일', '일요일']
    시간대_이름 = ['00~06시', '06~11시', '11~14시', '14~17시', '17~21시', '21~24시']

    # ✅ 휴무 추천일 예측 (매출이 가장 낮은 요일)
    요일평균 = df_train[요일_columns].mean()
    휴무추천일 = 요일평균.idxmin().replace('_매출_금액', '')

    # ✅ 휴식 추천 시간대 예측 (매출이 가장 낮은 시간대 3개)
    시간대평균 = df_train[시간대_columns].mean()
    휴식추천시간대 = 시간대평균.nsmallest(3).index.str.replace('_매출_금액', '').tolist()

    # ✅ 혼잡 예상 시간대 예측 (요일+시간대 기준으로 상위 3개)
    요일_시간대_조합 = []
    
    for 요일 in 요일_columns:
        for 시간대 in 시간대_columns:
            조합_컬럼 = f"{요일}_{시간대}"
            df_train[조합_컬럼] = df_train[요일] + df_train[시간대]  # 요일과 시간대 매출 합산
            요일_시간대_조합.append(조합_컬럼)

    혼잡_예상 = df_train[요일_시간대_조합].mean().nlargest(3)  # 상위 3개 조합
    혼잡예상시간대 = [f"{col.split('_')[0]} {col.split('_')[2]}" for col in 혼잡_예상.index]

    # ✅ 검증 데이터로 모델 학습 후 MAE, MSE 계산
    mae_list, mse_list = [], []
    for col in 요일_columns + 시간대_columns:
        X_train = df_train.drop(columns=요일_columns + 시간대_columns)
        y_train = df_train[col]
        if X_train.empty:
            continue
        
        model, mae, mse = train_model(X_train, y_train)
        mae_list.append(mae)
        mse_list.append(mse)

    평균_mae = np.mean(mae_list)
    평균_mse = np.mean(mse_list)

    print("\n📌 검증 데이터 평가 결과")
    print(f" - 평균 MAE: {평균_mae:.3f}")
    print(f" - 평균 MSE: {평균_mse:.3f}")

    return 휴무추천일, 휴식추천시간대, 혼잡예상시간대, 평균_mae, 평균_mse

# 실행
if __name__ == "__main__":
    df = pd.read_csv("C:\\Users\\m\\Desktop\\머신러닝 사용 데이터\\피처엔지니어링한통합데이터\\피처엔지니어링일단다한통합데이터.csv")

    상권명 = input("상권명을 입력하세요: ")
    업종명 = input("업종명을 입력하세요: ")
    
    df_train, df_test = preprocess_data(df, 상권명, 업종명)

    휴무추천일, 휴식추천시간대, 혼잡예상시간대, 평균_mae, 평균_mse = predict_sales(df_train, df_test)

    print("\n🔹 최종 예측 결과 🔹")
    print(f"휴무 추천일: {휴무추천일}")
    print(f"휴식 추천 시간대 (상위 3개): {', '.join(휴식추천시간대)}")
    print(f"혼잡 예상 시간대 (상위 3개): {', '.join(혼잡예상시간대)}")

ValueError: could not convert string to float: 'D'

In [32]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error

# 데이터 전처리 함수
def preprocess_data(df, 상권명, 업종명):
    df_filtered = df[(df['상권_코드_명'] == 상권명) & (df['서비스_업종_코드_명'] == 업종명)]
    df_train = df_filtered[df_filtered['기준_년분기_코드'] <= 20234]  # 2023년까지 학습
    df_test = df_filtered[df_filtered['기준_년분기_코드'] == 20241]  # 2024년 1분기로 검증
    return df_train, df_test

# 모델 학습 함수
def train_model(X, y):
    # 숫자형 데이터만 추출
    X_numeric = X.select_dtypes(include=[np.number])
    
    # 데이터셋이 비어 있는 경우 스킵
    if X_numeric.empty or y.empty:
        return None, None, None

    X_train, X_valid, y_train, y_valid = train_test_split(X_numeric, y, test_size=0.2, random_state=42)
    model = RandomForestRegressor(n_estimators=100, random_state=42)
    model.fit(X_train, y_train)

    y_valid_pred = model.predict(X_valid)
    y_valid_pred = np.nan_to_num(y_valid_pred, nan=np.mean(y_valid_pred))  # NaN이면 평균값으로 대체

    mae = mean_absolute_error(y_valid, y_valid_pred)
    mse = mean_squared_error(y_valid, y_valid_pred)

    return model, mae, mse

# 매출 예측 함수
def predict_sales(df_train, df_test):
    요일_columns = ['월요일_매출_금액', '화요일_매출_금액', '수요일_매출_금액', '목요일_매출_금액', '금요일_매출_금액', '토요일_매출_금액', '일요일_매출_금액']
    시간대_columns = ['시간대_00~06_매출_금액', '시간대_06~11_매출_금액', '시간대_11~14_매출_금액', '시간대_14~17_매출_금액', '시간대_17~21_매출_금액', '시간대_21~24_매출_금액']

    # ✅ 휴무 추천일 예측
    요일평균 = df_train[요일_columns].mean()
    휴무추천일 = 요일평균.idxmin().replace('_매출_금액', '')

    # ✅ 휴식 추천 시간대 예측
    시간대평균 = df_train[시간대_columns].mean()
    휴식추천시간대 = 시간대평균.nsmallest(3).index.str.replace('_매출_금액', '').tolist()

    # ✅ 혼잡 예상 시간대 예측
    요일_시간대_조합 = []
    for 요일 in 요일_columns:
        for 시간대 in 시간대_columns:
            조합_컬럼 = f"{요일}_{시간대}"
            df_train[조합_컬럼] = df_train[요일] + df_train[시간대]  # 요일과 시간대 매출 합산
            요일_시간대_조합.append(조합_컬럼)

    혼잡_예상 = df_train[요일_시간대_조합].mean().nlargest(3)  # 상위 3개 조합
    혼잡예상시간대 = [f"{col.split('_')[0]} {col.split('_')[2]}" for col in 혼잡_예상.index]

    # 검증 데이터로 모델 학습 후 MAE, MSE 계산
    mae_list, mse_list = [], []
    for col in 요일_columns + 시간대_columns:
        X_train = df_train.drop(columns=요일_columns + 시간대_columns)
        y_train = df_train[col]
        if X_train.empty:
            continue

        model, mae, mse = train_model(X_train, y_train)
        if model is None:
            continue
        mae_list.append(mae)
        mse_list.append(mse)

    평균_mae = np.mean(mae_list)
    평균_mse = np.mean(mse_list)

    print("\n📌 검증 데이터 평가 결과")
    print(f" - 평균 MAE: {평균_mae:.3f}")
    print(f" - 평균 MSE: {평균_mse:.3f}")

    return 휴무추천일, 휴식추천시간대, 혼잡예상시간대, 평균_mae, 평균_mse

# 실행
if __name__ == "__main__":
    df = pd.read_csv("C:\\Users\\m\\Desktop\\머신러닝 사용 데이터\\피처엔지니어링한통합데이터\\피처엔지니어링일단다한통합데이터.csv")

    상권명 = input("상권명을 입력하세요: ")
    업종명 = input("업종명을 입력하세요: ")
    
    df_train, df_test = preprocess_data(df, 상권명, 업종명)

    휴무추천일, 휴식추천시간대, 혼잡예상시간대, 평균_mae, 평균_mse = predict_sales(df_train, df_test)

    print("\n🔹 최종 예측 결과 🔹")
    print(f"휴무 추천일: {휴무추천일}")
    print(f"휴식 추천 시간대 (상위 3개): {', '.join(휴식추천시간대)}")
    print(f"혼잡 예상 시간대 (상위 3개): {', '.join(혼잡예상시간대)}")


📌 검증 데이터 평가 결과
 - 평균 MAE: 2626154.571
 - 평균 MSE: 12914396213630.758

🔹 최종 예측 결과 🔹
휴무 추천일: 일요일
휴식 추천 시간대 (상위 3개): 시간대_00~06, 시간대_06~11, 시간대_11~14
혼잡 예상 시간대 (상위 3개): 화요일 금액, 금요일 금액, 월요일 금액


In [59]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error

# 데이터 전처리 함수
def preprocess_data(df, 상권명, 업종명):
    df_filtered = df[(df['상권_코드_명'] == 상권명) & (df['서비스_업종_코드_명'] == 업종명)]
    df_train = df_filtered[df_filtered['기준_년분기_코드'] <= 20234]  # 2023년까지 학습
    df_test = df_filtered[df_filtered['기준_년분기_코드'] == 20242]  # 2024년 2분기로 검증
    return df_train, df_test

# 모델 학습 함수
def train_model(X, y):
    X_numeric = X.select_dtypes(include=[np.number])
    if X_numeric.empty or y.empty:
        return None, None, None

    X_train, X_valid, y_train, y_valid = train_test_split(X_numeric, y, test_size=0.2, random_state=42)
    model = RandomForestRegressor(n_estimators=100, random_state=42)
    model.fit(X_train, y_train)

    y_valid_pred = model.predict(X_valid)
    y_valid_pred = np.nan_to_num(y_valid_pred, nan=np.mean(y_valid_pred))

    mae = mean_absolute_error(y_valid, y_valid_pred)
    mse = mean_squared_error(y_valid, y_valid_pred)

    return model, mae, mse

# 매출 예측 함수
def predict_sales(df_train, df_test):
    요일_columns = ['월요일_매출_금액', '화요일_매출_금액', '수요일_매출_금액', '목요일_매출_금액', '금요일_매출_금액', '토요일_매출_금액', '일요일_매출_금액']
    시간대_columns = ['시간대_00~06_매출_금액', '시간대_06~11_매출_금액', '시간대_11~14_매출_금액', '시간대_14~17_매출_금액', '시간대_17~21_매출_금액', '시간대_21~24_매출_금액']

    # 휴무 추천일 예측
    요일평균 = df_train[요일_columns].mean()
    휴무추천일 = 요일평균.idxmin().replace('_매출_금액', '')

    # 휴식 추천 시간대 예측
    시간대평균 = df_train[시간대_columns].mean()
    휴식추천시간대 = 시간대평균.nsmallest(3).index.str.replace('시간대_', '').str.replace('_매출_금액', '').tolist()

    # 혼잡 예상 시간대 예측
    요일_시간대_조합 = []
    for 요일 in 요일_columns:
        for 시간대 in 시간대_columns:
            조합_컬럼 = f"{요일}_{시간대}"
            df_train[조합_컬럼] = df_train[요일] + df_train[시간대]
            요일_시간대_조합.append(조합_컬럼)

    혼잡_예상 = df_train[요일_시간대_조합].mean().nlargest(3)
    혼잡예상시간대 = []
    for col in 혼잡_예상.index:
        # Split column name to extract day and time period
        if '_시간대_' in col:
            day_part, time_part = col.split('_시간대_', 1)
            day = day_part.split('_')[0]  # e.g. "월요일"
            time_detail = time_part.split('_')[0]  # e.g. "00~06"
            # Format time period
            formatted_time = time_detail.replace('~', '시 ~ ') + '시'
            혼잡예상시간대.append(f"{day} {formatted_time}")

    # 검증 데이터로 모델 학습 후 MAE, MSE 계산
    mae_list, mse_list = [], []
    for col in 요일_columns + 시간대_columns:
        X_train = df_train.drop(columns=요일_columns + 시간대_columns)
        y_train = df_train[col]
        if X_train.empty:
            continue

        model, mae, mse = train_model(X_train, y_train)
        if model is None:
            continue
        mae_list.append(mae)
        mse_list.append(mse)

    평균_mae = np.mean(mae_list)
    평균_mse = np.mean(mse_list)

    print("\n📌 검증 데이터 평가 결과")
    print(f" - 평균 MAE: {평균_mae:.3f}")
    print(f" - 평균 MSE: {평균_mse:.3f}")

    return 휴무추천일, 휴식추천시간대, 혼잡예상시간대, 평균_mae, 평균_mse

# 실행 
if __name__ == "__main__":
    df = pd.read_csv("C:\\Users\\m\\Desktop\\머신러닝 사용 데이터\\피처엔지니어링한통합데이터\\피처엔지니어링일단다한통합데이터.csv")

    상권명 = input("상권명을 입력하세요: ")
    업종명 = input("업종명을 입력하세요: ")
    
    df_train, df_test = preprocess_data(df, 상권명, 업종명)

    휴무추천일, 휴식추천시간대, 혼잡예상시간대, 평균_mae, 평균_mse = predict_sales(df_train, df_test)

    print("\n🔹 최종 예측 결과 🔹")
    print(f"휴무 추천일: {휴무추천일}")
    print("휴식 추천 시간대 (상위 3개): " + ", ".join([f"{시간대.replace('~', '시 ~ ')}시" for 시간대 in 휴식추천시간대]))
    print("혼잡 예상 시간대 (상위 3개): " + ", ".join(혼잡예상시간대))


📌 검증 데이터 평가 결과
 - 평균 MAE: 1346502.127
 - 평균 MSE: 4772333158717.951

🔹 최종 예측 결과 🔹
휴무 추천일: 일요일
휴식 추천 시간대 (상위 3개): 00시 ~ 06시, 21시 ~ 24시, 17시 ~ 21시
혼잡 예상 시간대 (상위 3개): 목요일 06시 ~ 11시, 수요일 06시 ~ 11시, 화요일 06시 ~ 11시


In [ ]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sktime.forecasting.compose import make_reduction
from sktime.forecasting.model_selection import temporal_train_test_split
from sktime.forecasting.base import ForecastingHorizon

# 데이터 로드
df = pd.read_csv("C:\\Users\\m\\Desktop\\머신러닝 사용 데이터\\피처엔지니어링한통합데이터\\피처엔지니어링일단다한통합데이터.csv")

# 데이터 정렬 및 분기 정리
df = df.sort_values('기준_년분기_코드')
df['날짜'] = pd.to_datetime(df['기준_년분기_코드'].astype(str).str[:4] + 
                             pd.to_datetime((df['기준_년분기_코드'].astype(str).str[-1].astype(int) - 1) * 3 + 1, format='%m').dt.strftime('-%m-01'))

# 분기별 날짜 및 요일 정보 생성
results = []
for (quarter_code, group) in df.groupby('기준_년분기_코드'):
    start_date = group['날짜'].iloc[0]
    if start_date.month == 1:
        end_date = start_date + pd.DateOffset(months=3) - pd.DateOffset(days=1)
    else:
        end_date = pd.Timestamp(start_date.year, start_date.month + 2, 1) + pd.DateOffset(days=-1)

    dates_range = pd.date_range(start=start_date, end=end_date)
    day_counts = {day: sum(dates_range.day_name() == day) for day in ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']}

    for day, count in day_counts.items():
        group[day + '_count'] = count

    # 매출 데이터를 추정할 대상(날짜, 요일) 데이터 생성
    forecast_dates = pd.DataFrame({'날짜': dates_range})
    forecast_dates['요일'] = forecast_dates['날짜'].dt.day_name()
    forecast_dates['요일'] = forecast_dates['요일'].map({
        'Monday': '월', 'Tuesday': '화', 'Wednesday': '수',
        'Thursday': '목', 'Friday': '금', 'Saturday': '토', 'Sunday': '일'})
    forecast_dates['상권'] = group['상권_코드_명'].iloc[0]
    forecast_dates['업종'] = group['서비스_업종_코드_명'].iloc[0]

    # 학습 데이터 구성
    y_train = group.filter(regex='매출_금액').mean(axis=1)
    forecaster = make_reduction(RandomForestRegressor(), window_length=4, strategy="recursive")

    if len(y_train) < 5:  # 학습 데이터가 부족한 경우 스킵
        continue

    fh = ForecastingHorizon(np.arange(1, len(forecast_dates) + 1))

    try:
        forecaster.fit(y_train)
        y_pred = forecaster.predict(fh)
    except Exception as e:
        print(f"Error for {quarter_code}: {e}")
        continue

    forecast_dates['매출_금액'] = y_pred
    results.append(forecast_dates)

# 결과 데이터 병합 및 저장
final_df = pd.concat(results)
final_df.columns = ['날짜', '요일', '상권', '업종', '매출_금액']

# 결과 저장
final_df.to_csv("과거_추정_데이터.csv", index=False, encoding='utf-8-sig')
print("데이터 저장 완료: 과거_추정_데이터.csv")